# Coursera Capstone Project - Battle of the Neighborhoods: 
# Business Analysis on San Salvador (Metropolitan Area), El Salvador

## Introduction: Business Problem

This notebook is a study that analyzes every type of venue and will recommend the best locations to open a restaurant within the Metropolitan area of San Salvador (MASS). This area of El Salvador, contains 14 municipalities which are the following:

* San Salvador 
* Antiguo Cuscatlan
* San Marcos
* Soyapango
* Ilopango
* San Martin
* Tonacatepeque
* Ciudad Delgado
* Cuscatancingo
* Mejicanos
* Apopa
* Nejapa
* Ayutuxtepeque. 

From the center of San Salvador a radius of 15 KMs will set as a limit with the intention of covering the entire metropolitan area. The radius selection is due to lack of venues registered under the Foursquare database in the center of San Salvador.

## Data:

The analysis in this notebook will be based on the following requirements of the project to solve the question in hand:

* Types of existing restaurants in the area.
* Top types of restaurants and other venues in the area.



For practical purposes, the data was extracted from Foursquare Geolocation DB through the API calls. 
Foursquare is one of the world leaders on geospatial data and keeps its database fresh and constantly 
updated with the latest venues. **Problem solution** will come from the analysis on the process by which Data was extracted in raw form and processed into data frames, to be later clustered through machine learning algorithm (k-Means), this will provide a clear vision on what types of venues are in a certain area. Understanding what business types develop on certain areas of the city will supply patterns of how the Salvadoran society consumes or accesses a service.


####  Choosing Municipalities:

We will run several algorythms in the MASS, to later decide where will we focus our analysis.

In [1]:
##Library Imports
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import csv
import json
import requests
from geopy.geocoders import Nominatim
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium

In [2]:
#Location Assignation:
address = 'San Salvador, El Salvador'
geolocator = Nominatim(user_agent="sv_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of San Salvador are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of San Salvador are 13.6989939, -89.1914249.


In [3]:
#Foursquare API Credentials:
CLIENT_ID = 'CBR5UZU5JDYU2FAXPU1OQQWPYB4YVKBLJOTMIQTC3SDYO2CZ' # your Foursquare ID
CLIENT_SECRET = 'B3GM2R54H3S0X5QGLVDDO0EFOSQKPZ5G1TM4JEWTFJWUG2HV' # your Foursquare Secret
VERSION = '20200804' # Foursquare API version


print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: CBR5UZU5JDYU2FAXPU1OQQWPYB4YVKBLJOTMIQTC3SDYO2CZ
CLIENT_SECRET:B3GM2R54H3S0X5QGLVDDO0EFOSQKPZ5G1TM4JEWTFJWUG2HV


In [4]:
#Fourquare URL builder
limit = 130
radius = 15000
ll = 13.6758, -89.2895

# URL Creation, you can also test the API url directly from your explorer.
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    latitude, 
    longitude,
    VERSION,
    radius,
    limit)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=CBR5UZU5JDYU2FAXPU1OQQWPYB4YVKBLJOTMIQTC3SDYO2CZ&client_secret=B3GM2R54H3S0X5QGLVDDO0EFOSQKPZ5G1TM4JEWTFJWUG2HV&ll=13.6989939,-89.1914249&v=20200804&radius=15000&limit=130'

In [5]:
# The MASS area with Folium. Notice how the city is merged between the above mentioned municipalities.
map_sv = folium.Map(location=[latitude, longitude], zoom_start=12)
map_sv

#### In this map we are able to see how the MASS is composed of different municipalities that are neighboring from one another. We need to determine what will the Municipality be for a bussiness oportunity, in terms of the economic behavior of the areas.

In [6]:
#Calling all venues with our URL
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5f3888662b6bc220ae2232aa'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'San Salvador',
  'headerFullLocation': 'San Salvador',
  'headerLocationGranularity': 'city',
  'totalResults': 156,
  'suggestedBounds': {'ne': {'lat': 13.833994035000135,
    'lng': -89.0527314518288},
   'sw': {'lat': 13.563993764999864, 'lng': -89.3301183481712}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4c17c743fe5a76b0bc100115',
       'name': 'Puerto Marisco Aloha',
       'location': {'address': 'Calle El Progreso',
        'crossStreet': 'Colonia y Pasaje Avila',
        'lat': 13.696987293124769,
        'lng': -89.22242697386919,
        'labeledLatLngs

In [7]:
##Data standarization begins here.
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [8]:
#"Pandification" = converting the results into a Panda's Dataframe

address = results['response']['groups'][0]['items']
    
theadd = json_normalize(address) # flatten JSON
#Columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.formattedAddress', 'venue.location.lat','venue.location.lng']
thadd = theadd.loc[:, filtered_columns]

#Categories
theadd['venue.categories'] = theadd.apply(get_category_type, axis=1)

#Data cleansing - Columns
theadd.columns = [col.split(".")[-1] for col in theadd.columns]

theadd.head()

C:\Users\Andres\Anaconda3\lib\site-packages\pandas\core\indexing.py:1494: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


,count,items,referralId,categories,id,address,cc,city,country,crossStreet,...,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,name,count,groups,id
0,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4c17c743fe5a76b0bc100115-0,Seafood Restaurant,4c17c743fe5a76b0bc100115,Calle El Progreso,SV,San Salvador,El Salvador,Colonia y Pasaje Avila,...,"[Calle El Progreso (Colonia y Pasaje Avila), S...","[{'label': 'display', 'lat': 13.69698729312476...",13.696987,-89.222427,NaN,San Salvador,Puerto Marisco Aloha,0,[],NaN
1,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4d3f871cc5eaa1cdec5a9950-1,Latin American Restaurant,4d3f871cc5eaa1cdec5a9950,Calle Los Sisimiles,SV,San Salvador,El Salvador,Pasaje Chaparrastique,...,"[Calle Los Sisimiles (Pasaje Chaparrastique), ...","[{'label': 'display', 'lat': 13.71149980486681...",13.711500,-89.215874,1101,San Salvador,Clavo & Canela,0,[],NaN
2,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4dfd1f7f7d8b305080175384-2,Theater,4dfd1f7f7d8b305080175384,Boulevard De Los Heroes,SV,San Salvador,El Salvador,Calle Los Andes,...,"[Boulevard De Los Heroes (Calle Los Andes), Sa...","[{'label': 'display', 'lat': 13.70690459117522...",13.706905,-89.212308,NaN,San Salvador,Teatro Luis Poma,0,[],NaN
3,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4d8245517465cbff7c81cbed-3,Brazilian Restaurant,4d8245517465cbff7c81cbed,Hotel Real Intercontinental,SV,San Salvador,El Salvador,Calle Los Sisimiles,...,[Hotel Real Intercontinental (Calle Los Sisimi...,"[{'label': 'display', 'lat': 13.70767198152923...",13.707672,-89.213111,NaN,San Salvador,Faisca do Brasil Rodizio & Grill,0,[],39160804
4,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-5137c006e4b032c75a06c183-4,Sushi Restaurant,5137c006e4b032c75a06c183,Blvd Constitución #9,SV,San Salvador,El Salvador,NaN,...,"[Blvd Constitución #9, San Salvador, San Salva...","[{'label': 'display', 'lat': 13.71441214674336...",13.714412,-89.223640,NaN,San Salvador,La Susheria,0,[],49942704


#### The table above lets us see how the raw data is picked up from Foursquare, a cleansing process is required so we are able to visualize and analyse the results of the study.

In [9]:
#Data cleansing
categorias = theadd['categories'].to_list()
direccion = theadd['address'].to_list()
lati = theadd['lat'].to_list()
longi = theadd['lng'].to_list()
nombre = theadd['name'].to_list()

In [10]:
tabla = list(zip(nombre, categorias, direccion, lati, longi))

In [11]:
table_sv = pd.DataFrame(tabla, columns = ['Place', 'Category', 'Address', 'Latitude', 'Longitude'])

In [12]:
table_sv.head()

,Place,Category,Address,Latitude,Longitude
0,Puerto Marisco Aloha,Seafood Restaurant,Calle El Progreso,13.696987,-89.222427
1,Clavo & Canela,Latin American Restaurant,Calle Los Sisimiles,13.711500,-89.215874
2,Teatro Luis Poma,Theater,Boulevard De Los Heroes,13.706905,-89.212308
3,Faisca do Brasil Rodizio & Grill,Brazilian Restaurant,Hotel Real Intercontinental,13.707672,-89.213111
4,La Susheria,Sushi Restaurant,Blvd Constitución #9,13.714412,-89.223640


In [13]:
newsv = table_sv.dropna()
newsv.reset_index(drop = True)
newsv.head()

,Place,Category,Address,Latitude,Longitude
0,Puerto Marisco Aloha,Seafood Restaurant,Calle El Progreso,13.696987,-89.222427
1,Clavo & Canela,Latin American Restaurant,Calle Los Sisimiles,13.711500,-89.215874
2,Teatro Luis Poma,Theater,Boulevard De Los Heroes,13.706905,-89.212308
3,Faisca do Brasil Rodizio & Grill,Brazilian Restaurant,Hotel Real Intercontinental,13.707672,-89.213111
4,La Susheria,Sushi Restaurant,Blvd Constitución #9,13.714412,-89.223640


In [14]:
#understanding the quantity of venues
print('{} venues were returned by Foursquare.'.format(newsv.shape[0]))

73 venues were returned by Foursquare.


In [19]:
# Map of MASS venues with Folium
map_sv2 = folium.Map(location=[latitude, longitude], zoom_start=12)

# With this for we add the markers with the respected colors.
for lat, lng, neim, dire in zip(newsv['Latitude'], newsv['Longitude'], newsv['Place'], newsv['Address']):
    label = '{}, {}'.format(neim, dire)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_sv2)  
    
map_sv2

#### This map shows where the economic activity with web presence takes place. 

#### Now lets observe the activity in the nearby areas with this 15KM radius. This will show us what type of venues are in the areas in the SS area.

In [24]:
def getNearbyVenues(names, latitudes, longitudes, radius=15000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        #Get requests:
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        #return venue location
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [25]:
final_dfsv = newsv.rename(columns = {'Address':'Neighborhood'})

In [26]:
final_dfsv.head()

,Place,Category,Neighborhood,Latitude,Longitude
0,Puerto Marisco Aloha,Seafood Restaurant,Calle El Progreso,13.696987,-89.222427
1,Clavo & Canela,Latin American Restaurant,Calle Los Sisimiles,13.711500,-89.215874
2,Teatro Luis Poma,Theater,Boulevard De Los Heroes,13.706905,-89.212308
3,Faisca do Brasil Rodizio & Grill,Brazilian Restaurant,Hotel Real Intercontinental,13.707672,-89.213111
4,La Susheria,Sushi Restaurant,Blvd Constitución #9,13.714412,-89.223640


#### Data cleansing complete with these three lines of code, we standardize the use of the word "Neighborhood", since this could affect kNN Algorythm in the future. After this point no more data cleansing is required.


## Methodology

### Data extraction:

**Data extraction has three main sources:**

A.Jupyter Notebooks: 
    To process the entire study, we will use Jupyter Notebooks. 
    
B.Foursquare API:
    The Foursquare API will allow us to analyze neighborhoods and venues in each area of the city. A 15 km radius was chosen for data extraction
    
C.GDB data of the country, states, and municipalities:
    To understand how the municipalities are divided we require data extraction from GDB files. The GDB files were downloaded from the GeoSpatial

### Data Processing:

Standardization:
        
A. Foursquare API:
The raw data received from the API call a cleansing process is done through Python with its Pandas library for data manipulation functions and conversion of this raw information into data frame. 
            
        
B.	GDB:
        
The GDB files are already standardized, therefore no changes will occur to this type of file.
            
Information Generation:
    
After main data frames have been created, and GDB files have been loaded successfully, produced from the original data, new data frames and graphs will be generated, to examine different views which will allow us to analyze the problem and provide a solution.
    
Preliminary Data Analysis:
    
Individually counting the type of venue will reveal how common a venue could be, and although it bids the idea of commonness, the analysis would not be complete. Precision is required to understand the economic activity of the different municipalities in MASS. By now, it is evident where most of the economic activity in MASS occurs, three main municipalities are shown:

#### Antiguo Cuscatlán
#### Santa Tecla
#### San Salvador (North)
    
Foursquare works by the amount of times a person checks in a venue. Someone can subscribe venues, and if this venue becomes popular, then it will show up on top searches. If this study is focused from the city center of San Salvador (even by using 4KM radius), the amount of venues in the city center, is too low (57+- venues), the study could reflect faulty results. Therefore, this investigation will now redirect its focus to the three municipalities mentioned above; to determine, where in MASS could the Stakeholder open a restaurant. The process of analysis continues by extracting data of all nearby venues, of the one hundred sites originally summoned from Foursquare. This data will be processed to new data frames with values that are to be transformed from alphabetic to numeric, not only for standardization purposes, but also to generate new frames of data, with a frequency measuring the amount of times a type of venue concurs in a neighbourhood. This data set of frequencies can now tell us commonness of venue types. The last step of the process is to merge (inner join by neighbourhood) the original purified data frame, with the commonness data frame. This last information table shall be processed by kNN Machine Learning Algorithm who should provide a clear answer on what type of restaurant and best location for type. New data frames comparing the concurrence per cluster, will help the Stakeholder understand the economic activity in the area, and how to invest. Although there are other factors to consider when choosing a location, like price and security; this study will not take this data into account since more data is required from local authorities, who at the moment do not possess the technology infrastructure to extract this data from.

    
Visualization:
    
All stages of the data analysis are properly graphed with matrices representing the data frames, scatterplot maps representing geolocation of venues, other types of maps helping represent an idea of the area, these maps were made in Tableau.  

#### Now that the focus is directed into three main municipalities, the study will start from the beginning using Latitude and Longitudes from Google Maps, without accessing their API.

In [33]:
##Latitude, Langitude for Antiguo Cuscatlan
latitude_AC = 13.6734763
longitude_AC = -89.2416442
print('The geograpical coordinate of Antiguo Cuscatlan are {}, {}.'.format(latitude_AC, longitude_AC))

The geograpical coordinate of Antiguo Cuscatlan are 13.6734763, -89.2416442.


In [21]:
##Latitude, Langitude for Santa Tecla, starting from the center of the main city, Parque San Martin
ST_latitude = 13.676010
ST_longitude = -89.281296
print('The geograpical coordinate of ST are {}, {}.'.format(ST_latitude, ST_longitude))

The geograpical coordinate of ST are 13.67601, -89.281296.


In [22]:
##Latitude, Langitude for northern area of San Salvador from the Masferrer Roundabout
SS_latitude = 13.703484
SS_longitude = -89.248812
print('The geograpical coordinate of ST are {}, {}.'.format(SS_latitude, SS_longitude))

The geograpical coordinate of ST are 13.703484, -89.248812.


In [79]:
#Fourquare URL builder for Antiguo Cuscatlan
limit = 100
radius = 2000
ll_AC = 13.6734763, -89.2416442

# URL Creation, you can also test the API url directly from your explorer.
ACurl = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    latitude_AC, 
    longitude_AC,
    VERSION,
    radius,
    limit)
ACurl

'https://api.foursquare.com/v2/venues/explore?&client_id=CBR5UZU5JDYU2FAXPU1OQQWPYB4YVKBLJOTMIQTC3SDYO2CZ&client_secret=B3GM2R54H3S0X5QGLVDDO0EFOSQKPZ5G1TM4JEWTFJWUG2HV&ll=13.6734763,-89.2416442&v=20200816&radius=2000&limit=100'

In [80]:
#Fourquare URL builder for Santa Tecla
limit = 100
radius = 2000
ll_ST = 13.67601, -89.281296

# URL Creation, you can also test the API url directly from your explorer.
STurl = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    ST_latitude, 
    ST_longitude,
    VERSION,
    radius,
    limit)
STurl

'https://api.foursquare.com/v2/venues/explore?&client_id=CBR5UZU5JDYU2FAXPU1OQQWPYB4YVKBLJOTMIQTC3SDYO2CZ&client_secret=B3GM2R54H3S0X5QGLVDDO0EFOSQKPZ5G1TM4JEWTFJWUG2HV&ll=13.67601,-89.281296&v=20200816&radius=2000&limit=100'

In [81]:
#Fourquare URL builder for San Salvador (North)
limit = 100
radius = 2000
ll_SS = 13.703484, -89.248812

# URL Creation, you can also test the API url directly from your explorer.
SSurl = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    SS_latitude, 
    SS_longitude,
    VERSION,
    radius,
    limit)
SSurl

'https://api.foursquare.com/v2/venues/explore?&client_id=CBR5UZU5JDYU2FAXPU1OQQWPYB4YVKBLJOTMIQTC3SDYO2CZ&client_secret=B3GM2R54H3S0X5QGLVDDO0EFOSQKPZ5G1TM4JEWTFJWUG2HV&ll=13.703484,-89.248812&v=20200816&radius=2000&limit=100'

#### Map of Antiguo Cuscatlan:

In [35]:
map_AC1 = folium.Map(location=[latitude_AC, longitude_AC], zoom_start=15)
map_AC1

#### Map of Santa Tecla

In [28]:
map_ST1 = folium.Map(location=[ST_latitude, ST_longitude], zoom_start=15)
map_ST1

#### Map of San Salvador North

In [30]:
map_SS1 = folium.Map(location=[SS_latitude, SS_longitude], zoom_start=15)
map_SS1

#### Foursquare API calls to substract venues on the three different municipalities.

In [82]:
results_AC = requests.get(ACurl).json()
results_AC

{'meta': {'code': 200, 'requestId': '5f38a2826dd9a7231d873593'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'San Salvador',
  'headerFullLocation': 'San Salvador',
  'headerLocationGranularity': 'city',
  'totalResults': 154,
  'suggestedBounds': {'ne': {'lat': 13.691476318000019,
    'lng': -89.22315374573878},
   'sw': {'lat': 13.655476281999983, 'lng': -89.26013465426121}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4da884150cb65e9aae69f633',
       'name': 'Biscuit Factory Café Bistro',
       'location': {'address': 'Calle Principal Antiguo Cuscatlan',
        'lat': 13.67225,
        'lng': -89.240188,
        'labeledLatLngs': [{'label': 'display',
          'lat': 

In [83]:
results_ST = requests.get(STurl).json()
results_ST

{'meta': {'code': 200, 'requestId': '5f38a23ed333b03b331dfdef'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Santa Tecla',
  'headerFullLocation': 'Santa Tecla',
  'headerLocationGranularity': 'city',
  'totalResults': 120,
  'suggestedBounds': {'ne': {'lat': 13.694010018000018,
    'lng': -89.26280534679213},
   'sw': {'lat': 13.658009981999982, 'lng': -89.29978665320786}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '5023378fe4b0dc2236333c36',
       'name': 'Teklebab / The House of Kebab',
       'location': {'address': '5a. Ave norte',
        'crossStreet': '3a. calle oriente',
        'lat': 13.676083,
        'lng': -89.28461,
        'labeledLatLngs': [{'label': 'dis

In [84]:
results_SS = requests.get(SSurl).json()
results_SS

{'meta': {'code': 200, 'requestId': '5f38a3a1be3b4d17798a7df4'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'San Salvador',
  'headerFullLocation': 'San Salvador',
  'headerLocationGranularity': 'city',
  'totalResults': 167,
  'suggestedBounds': {'ne': {'lat': 13.721484018000018,
    'lng': -89.2303191869306},
   'sw': {'lat': 13.685483981999981, 'lng': -89.2673048130694}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4fbfde26e4b08acecad8cf6d',
       'name': 'Jägerhaus',
       'location': {'lat': 13.701781542350208,
        'lng': -89.24840641982195,
        'labeledLatLngs': [{'label': 'display',
          'lat': 13.701781542350208,
          'lng': -89.24840641982195}],

#### Data Standarization to be able to analyze the different venues and the different municipalities.

In [85]:
##Data standarization begins here.
def get_category_type_AC(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [86]:
##Data standarization begins here.
def get_category_type_ST(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [87]:
##Data standarization begins here.
def get_category_type_(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

#### New Dataframes created from the API calls:

In [88]:
address_AC = results_AC['response']['groups'][0]['items']
    
AC_venues = json_normalize(address_AC) # flatten JSON
#Columns
filtered_columns_AC = ['venue.name', 'venue.categories', 'venue.location.address', 'venue.location.lat','venue.location.lng']
AC_venues = AC_venues.loc[:, filtered_columns_AC]

#Categories
AC_venues['venue.categories'] = AC_venues.apply(get_category_type_AC, axis=1)

#Data cleansing - Columns
AC_venues.columns = [col.split(".")[-1] for col in AC_venues.columns]

AC_venues.head()

,name,categories,address,lat,lng
0,Biscuit Factory Café Bistro,Café,Calle Principal Antiguo Cuscatlan,13.672250,-89.240188
1,The Donut Place,Donut Shop,NaN,13.675482,-89.244515
2,4 Monkeys Café,Breakfast Spot,NaN,13.673681,-89.238330
3,Sushi King,Sushi Restaurant,Ave. Rio Amazonas,13.677708,-89.238621
4,Oasis: Jugos y Licuados,Juice Bar,Ave. Antiguo Cuscatlan,13.677677,-89.239972


In [90]:
address_ST = results_ST['response']['groups'][0]['items']
    
ST_venues = json_normalize(address_ST) # flatten JSON
#Columns
filtered_columns_ST = ['venue.name', 'venue.categories', 'venue.location.address', 'venue.location.lat','venue.location.lng']
ST_venues = ST_venues.loc[:, filtered_columns_ST]

#Categories
ST_venues['venue.categories'] = ST_venues.apply(get_category_type, axis=1)

#Data cleansing - Columns
ST_venues.columns = [col.split(".")[-1] for col in ST_venues.columns]

ST_venues.head()

,name,categories,address,lat,lng
0,Teklebab / The House of Kebab,Turkish Restaurant,5a. Ave norte,13.676083,-89.284610
1,La Grotta Crossfit,Gym / Fitness Center,"5ta Calle Oriente 3-2, Santa Tecla",13.677385,-89.285073
2,McDonald's,Burger Joint,NaN,13.671935,-89.276634
3,Boost,Juice Bar,Plaza Merliot,13.678574,-89.276152
4,Paseo El Carmen,Plaza,1° Calle Pte.,13.675393,-89.286953


In [91]:
address_SS = results_SS['response']['groups'][0]['items']
    
SS_venues = json_normalize(address_SS) # flatten JSON
#Columns
filtered_columns_SS = ['venue.name', 'venue.categories', 'venue.location.address', 'venue.location.lat','venue.location.lng']
SS_venues = SS_venues.loc[:, filtered_columns_SS]

#Categories
SS_venues['venue.categories'] = SS_venues.apply(get_category_type, axis=1)

#Data cleansing - Columns
SS_venues.columns = [col.split(".")[-1] for col in SS_venues.columns]

SS_venues.head()

,name,categories,address,lat,lng
0,Jägerhaus,German Restaurant,NaN,13.701782,-89.248406
1,Sento Luxury Spa & Salon,Spa,3a. calle poniente # 4817,13.704912,-89.244470
2,Vanilla Spoon,Frozen Yogurt Shop,"Centro Comercial Masferrrer, frente a Burger K...",13.703156,-89.248551
3,Jugueton Masferrer,Toy / Game Store,Plaza Masferrer,13.704420,-89.248532
4,Crêpe Lovers,Dessert Shop,Calle masferrer,13.705113,-89.249201


In [92]:
newAC = AC_venues.dropna()

In [110]:
newAC.reset_index(drop = True, inplace = True)

In [111]:
the_final_AC = newAC.rename(columns = {'name':'Venue', 'categories':'Category', 'address':'Neighborhood', 'lat':'Latitude', 'lng':'Longitude'})

In [112]:
the_final_AC.head()

,Venue,Category,Neighborhood,Latitude,Longitude
0,Biscuit Factory Café Bistro,Café,Calle Principal Antiguo Cuscatlan,13.672250,-89.240188
1,Sushi King,Sushi Restaurant,Ave. Rio Amazonas,13.677708,-89.238621
2,Oasis: Jugos y Licuados,Juice Bar,Ave. Antiguo Cuscatlan,13.677677,-89.239972
3,Green Dreams,Tea Room,hacienda los miranda,13.677662,-89.244861
4,Jardín Botánico La Laguna,Garden,Antiguo cuscatlan,13.670026,-89.246986


In [193]:
the_final_AC['Category'].value_counts()

Coffee Shop                      6
Café                             4
Shopping Mall                    4
Pizza Place                      3
Italian Restaurant               3
Hotel                            3
Sushi Restaurant                 3
Bakery                           3
Restaurant                       3
Latin American Restaurant        3
Department Store                 2
Juice Bar                        2
Salad Place                      2
Plaza                            1
Soccer Stadium                   1
Market                           1
French Restaurant                1
Museum                           1
Movie Theater                    1
Garden Center                    1
Frozen Yogurt Shop               1
Sports Bar                       1
Lounge                           1
American Restaurant              1
Mexican Restaurant               1
Tea Room                         1
Asian Restaurant                 1
Gym                              1
Wings Joint         

In [196]:
the_final_AC['Category'].replace({'Coffee Shop': 'Café'}, inplace = True)

In [197]:
the_final_AC['Category'].value_counts()

Café                             10
Shopping Mall                     4
Pizza Place                       3
Hotel                             3
Italian Restaurant                3
Sushi Restaurant                  3
Bakery                            3
Restaurant                        3
Latin American Restaurant         3
Department Store                  2
Juice Bar                         2
Salad Place                       2
Plaza                             1
Market                            1
Soccer Stadium                    1
French Restaurant                 1
Museum                            1
Frozen Yogurt Shop                1
Movie Theater                     1
Lounge                            1
Tea Room                          1
American Restaurant               1
Sports Bar                        1
Garden Center                     1
Mexican Restaurant                1
Asian Restaurant                  1
Gym                               1
Wings Joint                 

In [94]:
newST = ST_venues.dropna()

In [113]:
newST.reset_index(drop = True, inplace = True)

In [114]:
newST.rename(columns = {'name':'Venue', 'categories':'Category', 'address':'Neighborhood', 'lat':'Latitude', 'lng':'Longitude'}, inplace = True)

C:\Users\Andres\Anaconda3\lib\site-packages\pandas\core\frame.py:4025: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


In [115]:
newST.head()

,Venue,Category,Neighborhood,Latitude,Longitude
0,Teklebab / The House of Kebab,Turkish Restaurant,5a. Ave norte,13.676083,-89.284610
1,La Grotta Crossfit,Gym / Fitness Center,"5ta Calle Oriente 3-2, Santa Tecla",13.677385,-89.285073
2,Boost,Juice Bar,Plaza Merliot,13.678574,-89.276152
3,Paseo El Carmen,Plaza,1° Calle Pte.,13.675393,-89.286953
4,Museo Municipal Tecleno MUTE,Art Gallery,7 avenida norte,13.673568,-89.283809


In [96]:
newSS = SS_venues.dropna()

In [117]:
newSS.reset_index(drop = True, inplace = True)

In [118]:
newSS.rename(columns = {'name':'Venue', 'categories':'Category', 'address':'Neighborhood', 'lat':'Latitude', 'lng':'Longitude'}, inplace = True)

In [244]:
newST['Category'].value_counts()

Pizza Place                  8
Bar                          6
Latin American Restaurant    5
Seafood Restaurant           4
Bakery                       3
Sandwich Place               3
Mexican Restaurant           3
Café                         3
Art Gallery                  2
Chinese Restaurant           2
Shopping Mall                2
Sports Bar                   1
Plaza                        1
Pharmacy                     1
Wine Bar                     1
Coffee Shop                  1
Restaurant                   1
Department Store             1
Nightclub                    1
Ice Cream Shop               1
Salad Place                  1
Turkish Restaurant           1
Juice Bar                    1
Fried Chicken Joint          1
Other Great Outdoors         1
Gym / Fitness Center         1
Wings Joint                  1
Gift Shop                    1
Beach                        1
Gourmet Shop                 1
Pool                         1
Beer Garden                  1
Electron

In [245]:
newSS['Category'].value_counts()

Pizza Place                      4
Chinese Restaurant               4
Italian Restaurant               4
Mexican Restaurant               3
Hotel                            3
Café                             3
Restaurant                       3
Shopping Mall                    2
Coffee Shop                      2
French Restaurant                2
Sushi Restaurant                 2
Spa                              2
Liquor Store                     2
Japanese Restaurant              2
German Restaurant                2
Juice Bar                        2
Bakery                           2
Fast Food Restaurant             2
Seafood Restaurant               2
Sandwich Place                   1
Sports Bar                       1
Plaza                            1
Nightclub                        1
Pharmacy                         1
Ice Cream Shop                   1
Frozen Yogurt Shop               1
Supermarket                      1
Bar                              1
Salad Place         

In [119]:
newSS.head()

,Venue,Category,Neighborhood,Latitude,Longitude
0,Sento Luxury Spa & Salon,Spa,3a. calle poniente # 4817,13.704912,-89.244470
1,Vanilla Spoon,Frozen Yogurt Shop,"Centro Comercial Masferrrer, frente a Burger K...",13.703156,-89.248551
2,Jugueton Masferrer,Toy / Game Store,Plaza Masferrer,13.704420,-89.248532
3,Crêpe Lovers,Dessert Shop,Calle masferrer,13.705113,-89.249201
4,Taquería Los Tapatíos,Mexican Restaurant,Paseo General Escalon,13.703504,-89.247850


### Maps of the different venues, and different municipalities:

#### Map of Antiguo Cuscatlan venues, 2KM radius from the center of the municipality.

In [199]:
map_AC2 = folium.Map(location=[latitude_AC, longitude_AC], zoom_start=15)

# With this for we add the markers with the respected colors.
for lat, lng, neim, dire in zip(the_final_AC['Latitude'], the_final_AC['Longitude'], the_final_AC['Venue'], the_final_AC['Neighborhood']):
    label = '{}, {}'.format(neim, dire)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_AC2)  
    
map_AC2

#### Map of Santa Tecla venues, 2KM radius from the center of the municipality.

In [126]:
map_st2 = folium.Map(location=[ST_latitude, ST_longitude], zoom_start=15)

# With this for we add the markers with the respected colors.
for lat, lng, neim, dire in zip(newST['Latitude'], newST['Longitude'], newST['Venue'], newST['Neighborhood']):
    label = '{}, {}'.format(neim, dire)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='green',
        fill=True,
        fill_color='#86cc31',
        fill_opacity=0.7,
        parse_html=False).add_to(map_st2)  
    
map_st2

#### Map of San Salvador North venues, 2KM radius from the center of the Masferrer Roundabout.

In [127]:
map_ss2 = folium.Map(location=[SS_latitude, SS_longitude], zoom_start=15)

# With this for we add the markers with the respected colors.
for lat, lng, neim, dire in zip(newSS['Latitude'], newSS['Longitude'], newSS['Venue'], newSS['Neighborhood']):
    label = '{}, {}'.format(neim, dire)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#cc3139',
        fill_opacity=0.7,
        parse_html=False).add_to(map_ss2)  
    
map_ss2

### The analysis requires that we examine all nearby venues, to pull out a winner among the Municipalities.

In [200]:
# Nearby venues for Antiguo Cuscatlan:
def getNearbyVenues_AC(names, latitudes, longitudes, radius=2000):
    
    venues_list_AC=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        ACurl = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        #Get requests:
        resultsAC = requests.get(ACurl).json()["response"]['groups'][0]['items']
        
        #return venue location
        venues_list_AC.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in resultsAC])

    nearby_venues_AC = pd.DataFrame([item for venue_list_AC in venues_list_AC for item in venue_list_AC])
    nearby_venues_AC.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues_AC)

In [138]:
# Nearby venues for Santa Tecla:
def getNearbyVenues_ST(names, latitudes, longitudes, radius=2000):
    
    venues_list_ST=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        STurl = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        #Get requests:
        resultsST = requests.get(STurl).json()["response"]['groups'][0]['items']
        
        #return venue location
        venues_list_ST.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in resultsST])

    nearby_venues_ST = pd.DataFrame([item for venue_list_ST in venues_list_ST for item in venue_list_ST])
    nearby_venues_ST.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues_ST)

In [141]:
# Nearby venues for San Salvador (North):
def getNearbyVenues_SS(names, latitudes, longitudes, radius=2000):
    
    venues_list_SS=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        SSurl = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        #Get requests:
        resultsSS = requests.get(SSurl).json()["response"]['groups'][0]['items']
        
        #return venue location
        venues_list_SS.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in resultsSS])

    nearby_venues_SS = pd.DataFrame([item for venue_list_SS in venues_list_SS for item in venue_list_SS])
    nearby_venues_SS.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues_SS)

In [201]:
radius = 2000
LIMIT = 100

ac_venues = getNearbyVenues_AC(names=the_final_AC['Neighborhood'],
                                   latitudes=the_final_AC['Latitude'],
                                   longitudes=the_final_AC['Longitude'],
                                  )

Calle Principal Antiguo Cuscatlan
Ave. Rio Amazonas
Ave. Antiguo Cuscatlan
hacienda los miranda
Antiguo cuscatlan
Zona Industrial La Laguna
Clle. del Mediterráneo
C.C Multiplaza
1a Calle Pte block P #21
Calle Cuscatlán Poniente
Carr. Panamericana
Multiplaza
C.C Multiplaza
Antiguo Cuscatlan
Carretera Panamericana
1ra Calle Oriente
Calle Circunvalación pasaje #2 Condominio Victoria. Lomas De San Francisco
Peatonal de la UCA
Antiguo Cuscatlan
Carretera Panamericana
Centro Comercial La Gran Via Local 11-B
Blvd. Santa Elena
Antiguo Cuscatlan
Antiguo Cuscatlan
Centro Comercial Multiplaza
Calle las Rosas
Multiplaza El Salvador
Avenida la Revolución
Centro Comercial Multiplaza
Antiguo cuscatlan
Boulevard  El Hipodromo
1° Calle Oriente
C.C La Gran Via
Col. Lomas de San Francisco
Urb. Madre Selva, Blvd. Sur
SOHO Las Cascadas
Boulevard Los Próceres
Calle #1
Calle Panamericana
Calle Del Mediterraneo
Centro Comercial La Gran Vía
Plaza Madero
La Gran Via
Calle del Mediterráneo #17
Centro de Estilo d

In [146]:
radius = 2000
LIMIT = 100

ST_venues = getNearbyVenues_ST(names=newST['Neighborhood'],
                                   latitudes=newST['Latitude'],
                                   longitudes=newST['Longitude'],
                                  )

5a. Ave norte
5ta Calle Oriente 3-2, Santa Tecla
Plaza Merliot
1° Calle Pte.
7 avenida norte
Ciudad Merliot, Santa Teresa
Centro Comercial Chiltiupán
17 Ave Norte, #8, Col. Santa Teresa
Carretera Panamericana
Paseo El Carmen
7a. Avenida Norte
1era. Calle Oriente 3-7
17 avenida norte
Calle Chiltiupan
Calle Chiltiupán
Calle Chiltiupán
Centro Comercial La Skina
Calle José Ciriaco López
1 Calle Poniente, local 2-A
Centro Comercial Plaza Borbon Local 1
Paseo El Carmen, entre 3 y 5 Av. Norte
Pinares de Suiza
Texaco Chiltiupan
Ciudad Merliot
Calle Chiltiupan
CC La Skina
C.C Plaza Merliot
Centro Comercial La Skina
Calle Don Bosco #3-2
CALLE CHILTIUPAN Y 21 AVENIDA NORTE ,FRENTE A PLAZA MERLIOT, SANTA TECLA
Av. Manuel Gallardo
17 Av. Norte Calle La Sabana No 23
16 Avenida Norte
Paseo Del Carmen
Paseo El Carmen
boulevard orden de malta santa elena, antiguo cuscatlan
Paseo El Carmen
Ciudad Merliot
17 Ave Norte #17
Calle Chiltiupan
1 Calle Poniente
La Libertad, El Salvador
Primera Calle Oriente No

In [147]:
radius = 2000
LIMIT = 100

Ss_venues = getNearbyVenues_SS(names=newSS['Neighborhood'],
                                   latitudes=newSS['Latitude'],
                                   longitudes=newSS['Longitude'],
                                  )

3a. calle poniente # 4817
Centro Comercial Masferrrer, frente a Burger King Masferrer
Plaza Masferrer
Calle masferrer
Paseo General Escalon
Colonia Escalon
Calle Maximo Jerez
Colonia Escalon
Plaza Las Azaleas
Centro Comercial Las Azaleas
81 Av norte y 7a calle poniente, #4119
Centro Comercial Los Castaños, Av. Masferrer.
Avenida Masferrer Norte, Comercial Barcelona
Paseo General Escalon
Torre Futura
Plaza L'Abero, Local 1A
87 Av. Sur y Av. Los Almendros.
Calle y Colonia La Mascota No.942
Centro comercial Los Castaños
Plaza Barcelona, Colonia Escalón
Calle El Mirador-Escalón
89 Av Norte Escalon
3a Calle Poniente #5254 Lomas Verdes, Colonia Escalón
Calle La Mascota #7-A, Col. Maquilishuat
Col. Escalón 85 Av. Norte Pie. Dordely # 4407
Plaza Masferrer
Blv. El Hipodromo #582, San Benito
Calle La Mascota #803
Centro Comercial Los Castaños Av. Alberto Masferrer
Plaza Azaleas
9 C poniente bis
91 Avenida Norte #532
Final Paseo General Escalón
Ave Masferrer
Plaza Los Castaños
Plaza Futura
79 Av 

## Analysis:

### A radius of 2Kms, from each municipality center in Antiguo Cuscatlan, Santa Tecla; for San Salvador a Neighborhood known as Escalon Norte was chosen for its center, since the study reflects more economic movement on this area.  The behavior of the areas has been set to extract all nearby venues:

In [247]:
ac_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Calle Principal Antiguo Cuscatlan,13.67225,-89.240188,4 Monkeys Café,13.673681,-89.238330,Breakfast Spot
1,Calle Principal Antiguo Cuscatlan,13.67225,-89.240188,Biscuit Factory Café Bistro,13.672250,-89.240188,Café
2,Calle Principal Antiguo Cuscatlan,13.67225,-89.240188,Sushi King,13.677708,-89.238621,Sushi Restaurant
3,Calle Principal Antiguo Cuscatlan,13.67225,-89.240188,The Donut Place,13.675482,-89.244515,Donut Shop
4,Calle Principal Antiguo Cuscatlan,13.67225,-89.240188,Oasis: Jugos y Licuados,13.677677,-89.239972,Juice Bar


In [203]:
# one hot encoding in order to tranform into binary numeric values
AC_onehot = pd.get_dummies(ac_venues[['Venue Category']], prefix="", prefix_sep="")

# create column neigborhood and insert into dataframe
AC_onehot['Neighborhood'] = ac_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [AC_onehot.columns[-1]] + list(AC_onehot.columns[:-1])
AC_onehot = AC_onehot[fixed_columns]

AC_onehot.head()

,Neighborhood,American Restaurant,Arcade,Argentinian Restaurant,Art Gallery,Asian Restaurant,Bakery,Bar,Beach,Beer Bar,...,Sushi Restaurant,Taco Place,Tea Room,Theater,Theme Park,Toy / Game Store,Vegetarian / Vegan Restaurant,Veterinarian,Vietnamese Restaurant,Wings Joint
0,Calle Principal Antiguo Cuscatlan,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Calle Principal Antiguo Cuscatlan,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Calle Principal Antiguo Cuscatlan,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
3,Calle Principal Antiguo Cuscatlan,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Calle Principal Antiguo Cuscatlan,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [149]:
ST_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,5a. Ave norte,13.676083,-89.28461,Teklebab / The House of Kebab,13.676083,-89.284610,Turkish Restaurant
1,5a. Ave norte,13.676083,-89.28461,La Grotta Crossfit,13.677385,-89.285073,Gym / Fitness Center
2,5a. Ave norte,13.676083,-89.28461,Paseo El Carmen,13.675393,-89.286953,Plaza
3,5a. Ave norte,13.676083,-89.28461,Museo Municipal Tecleno MUTE,13.673568,-89.283809,Art Gallery
4,5a. Ave norte,13.676083,-89.28461,"Maurita, Mariscos Y Cocteles",13.675426,-89.288888,Seafood Restaurant


In [152]:
ST_onehot = pd.get_dummies(ST_venues[['Venue Category']], prefix="", prefix_sep="")

ST_onehot['Neighborhood'] = ST_venues['Neighborhood'] 

fixed_columns_ST = [ST_onehot.columns[-1]] + list(ST_onehot.columns[:-1])
ST_onehot = ST_onehot[fixed_columns_ST]

ST_onehot.head()

,Neighborhood,American Restaurant,Argentinian Restaurant,Art Gallery,Asian Restaurant,Athletics & Sports,Auto Garage,BBQ Joint,Bakery,Bar,...,Spanish Restaurant,Sports Bar,Steakhouse,Supermarket,Sushi Restaurant,Tea Room,Turkish Restaurant,Water Park,Wine Bar,Wings Joint
0,5a. Ave norte,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1,5a. Ave norte,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,5a. Ave norte,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,5a. Ave norte,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5a. Ave norte,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [150]:
Ss_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,3a. calle poniente # 4817,13.704912,-89.24447,Sento Luxury Spa & Salon,13.704912,-89.244470,Spa
1,3a. calle poniente # 4817,13.704912,-89.24447,Kamakura,13.708372,-89.244188,Japanese Restaurant
2,3a. calle poniente # 4817,13.704912,-89.24447,Plaza Futura,13.708232,-89.241069,Plaza
3,3a. calle poniente # 4817,13.704912,-89.24447,La Herradura Escalon,13.704272,-89.241937,Bar
4,3a. calle poniente # 4817,13.704912,-89.24447,Beto's Escalón,13.704369,-89.239815,Seafood Restaurant


In [153]:
Ss_onehot = pd.get_dummies(Ss_venues[['Venue Category']], prefix="", prefix_sep="")

Ss_onehot['Neighborhood'] = Ss_venues['Neighborhood'] 

fixed_columns_Ss = [Ss_onehot.columns[-1]] + list(Ss_onehot.columns[:-1])
Ss_onehot = Ss_onehot[fixed_columns_Ss]

Ss_onehot.head()

,Neighborhood,Accessories Store,American Restaurant,Arcade,Argentinian Restaurant,Art Gallery,Asian Restaurant,BBQ Joint,Bakery,Bar,...,Sushi Restaurant,Taco Place,Tea Room,Theater,Toy / Game Store,Vegetarian / Vegan Restaurant,Veterinarian,Video Game Store,Wings Joint,Yoga Studio
0,3a. calle poniente # 4817,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,3a. calle poniente # 4817,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3a. calle poniente # 4817,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3a. calle poniente # 4817,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,3a. calle poniente # 4817,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [204]:
AC_grouped = AC_onehot.groupby('Neighborhood').mean().reset_index()
AC_grouped

,Neighborhood,American Restaurant,Arcade,Argentinian Restaurant,Art Gallery,Asian Restaurant,Bakery,Bar,Beach,Beer Bar,...,Sushi Restaurant,Taco Place,Tea Room,Theater,Theme Park,Toy / Game Store,Vegetarian / Vegan Restaurant,Veterinarian,Vietnamese Restaurant,Wings Joint
0,1a Calle Pte block P #21,0.010000,0.0100,0.000000,0.000,0.000000,0.030000,0.000000,0.000,0.010000,...,0.030000,0.00,0.010000,0.0000,0.00,0.010000,0.010000,0.00,0.00,0.020000
1,1ra Calle Oriente,0.010000,0.0000,0.000000,0.000,0.010000,0.030000,0.000000,0.000,0.010000,...,0.030000,0.00,0.010000,0.0000,0.00,0.020000,0.010000,0.00,0.00,0.010000
2,1° Calle Oriente,0.010000,0.0000,0.000000,0.000,0.010000,0.030000,0.000000,0.000,0.010000,...,0.030000,0.00,0.010000,0.0000,0.00,0.020000,0.010000,0.00,0.00,0.010000
3,Antigua Vía Ferrocarril,0.010000,0.0000,0.000000,0.010,0.010000,0.030000,0.000000,0.000,0.010000,...,0.030000,0.00,0.010000,0.0100,0.00,0.000000,0.010000,0.00,0.00,0.010000
4,Antiguo Cuscatlan,0.007500,0.0075,0.002500,0.005,0.007500,0.032500,0.000000,0.000,0.010000,...,0.020000,0.00,0.010000,0.0025,0.00,0.007500,0.010000,0.00,0.00,0.020000
5,Antiguo cuscatlan,0.015000,0.0050,0.005000,0.005,0.005000,0.030000,0.005000,0.000,0.010000,...,0.020000,0.00,0.010000,0.0050,0.00,0.000000,0.010000,0.00,0.00,0.015000
6,Av. La Revolucion 184,0.000000,0.0100,0.000000,0.010,0.000000,0.030000,0.010000,0.000,0.010000,...,0.010000,0.01,0.010000,0.0100,0.00,0.010000,0.000000,0.00,0.00,0.020000
7,Av. Rio Amazonas #3,0.010000,0.0100,0.000000,0.010,0.000000,0.020000,0.000000,0.000,0.010000,...,0.010000,0.00,0.010000,0.0100,0.00,0.000000,0.010000,0.00,0.00,0.020000
8,Ave. Antiguo Cuscatlan,0.010000,0.0100,0.000000,0.010,0.000000,0.020000,0.000000,0.000,0.010000,...,0.020000,0.00,0.010000,0.0100,0.00,0.000000,0.010000,0.00,0.00,0.020000
9,"Ave. La Revolucion, Col. San Benito",0.000000,0.0100,0.000000,0.010,0.000000,0.030000,0.010000,0.000,0.010000,...,0.010000,0.01,0.010000,0.0100,0.00,0.010000,0.000000,0.00,0.00,0.020000


In [156]:
ST_grouped = ST_onehot.groupby('Neighborhood').mean().reset_index()
ST_grouped

,Neighborhood,American Restaurant,Argentinian Restaurant,Art Gallery,Asian Restaurant,Athletics & Sports,Auto Garage,BBQ Joint,Bakery,Bar,...,Spanish Restaurant,Sports Bar,Steakhouse,Supermarket,Sushi Restaurant,Tea Room,Turkish Restaurant,Water Park,Wine Bar,Wings Joint
0,1 Calle Poniente,0.010989,0.010989,0.021978,0.010989,0.000000,0.000000,0.021978,0.032967,0.131868,...,0.010989,0.010989,0.000000,0.010989,0.000000,0.00,0.010989,0.000000,0.010989,0.010989
1,"1 Calle Poniente, local 2-A",0.010101,0.010101,0.020202,0.010101,0.000000,0.000000,0.020202,0.030303,0.121212,...,0.010101,0.010101,0.000000,0.010101,0.000000,0.00,0.010101,0.000000,0.010101,0.010101
2,15 A Avenida Norte,0.010000,0.010000,0.020000,0.000000,0.000000,0.000000,0.010000,0.040000,0.150000,...,0.010000,0.010000,0.000000,0.010000,0.010000,0.00,0.010000,0.000000,0.010000,0.000000
3,16 Av Norte,0.013158,0.000000,0.026316,0.013158,0.000000,0.000000,0.013158,0.026316,0.157895,...,0.013158,0.013158,0.000000,0.013158,0.000000,0.00,0.013158,0.000000,0.013158,0.013158
4,16 Avenida Norte,0.013158,0.000000,0.026316,0.013158,0.000000,0.000000,0.013158,0.026316,0.157895,...,0.013158,0.013158,0.000000,0.013158,0.000000,0.00,0.013158,0.000000,0.013158,0.013158
5,17 Av. Norte Calle La Sabana No 23,0.010000,0.010000,0.020000,0.000000,0.000000,0.000000,0.010000,0.040000,0.150000,...,0.010000,0.010000,0.000000,0.010000,0.010000,0.00,0.010000,0.000000,0.010000,0.000000
6,17 Av. Sur,0.010000,0.010000,0.020000,0.000000,0.010000,0.000000,0.010000,0.060000,0.130000,...,0.010000,0.010000,0.000000,0.030000,0.000000,0.00,0.010000,0.010000,0.010000,0.000000
7,17 Ave Norte #17,0.010000,0.010000,0.020000,0.000000,0.000000,0.000000,0.010000,0.040000,0.150000,...,0.010000,0.010000,0.000000,0.010000,0.010000,0.00,0.010000,0.000000,0.010000,0.000000
8,"17 Ave Norte, #8, Col. Santa Teresa",0.010000,0.010000,0.020000,0.000000,0.000000,0.000000,0.010000,0.040000,0.150000,...,0.010000,0.010000,0.000000,0.020000,0.010000,0.00,0.010000,0.000000,0.010000,0.000000
9,17 avenida norte,0.010000,0.010000,0.020000,0.000000,0.000000,0.000000,0.010000,0.040000,0.150000,...,0.010000,0.010000,0.000000,0.010000,0.010000,0.00,0.010000,0.000000,0.010000,0.000000


In [157]:
Ss_grouped = Ss_onehot.groupby('Neighborhood').mean().reset_index()
Ss_grouped

,Neighborhood,Accessories Store,American Restaurant,Arcade,Argentinian Restaurant,Art Gallery,Asian Restaurant,BBQ Joint,Bakery,Bar,...,Sushi Restaurant,Taco Place,Tea Room,Theater,Toy / Game Store,Vegetarian / Vegan Restaurant,Veterinarian,Video Game Store,Wings Joint,Yoga Studio
0,"11 Calle Poniente, Calle El Mirador",0.000,0.010000,0.00,0.010000,0.00,0.010000,0.00,0.03,0.01,...,0.010000,0.020000,0.00,0.00,0.01,0.010000,0.00,0.00,0.020000,0.01
1,"3a Calle Poniente #5254 Lomas Verdes, Colonia ...",0.000,0.000000,0.00,0.010000,0.01,0.000000,0.01,0.02,0.01,...,0.020000,0.010000,0.00,0.01,0.01,0.010000,0.00,0.00,0.020000,0.01
2,3a. Calle Poniente,0.010,0.000000,0.00,0.010000,0.01,0.000000,0.00,0.02,0.01,...,0.030000,0.000000,0.00,0.01,0.01,0.010000,0.01,0.00,0.020000,0.00
3,3a. calle poniente # 4817,0.000,0.010000,0.00,0.010000,0.01,0.000000,0.01,0.02,0.01,...,0.010000,0.010000,0.00,0.01,0.01,0.010000,0.00,0.00,0.020000,0.01
4,77 Av Norte #346 Col. Escalon,0.000,0.010000,0.00,0.010000,0.01,0.000000,0.00,0.02,0.02,...,0.030000,0.010000,0.00,0.01,0.01,0.010000,0.01,0.00,0.020000,0.00
5,"79 Av Norte Y 3 Calle Poniente,colonia Escalon",0.000,0.010000,0.00,0.000000,0.01,0.000000,0.00,0.02,0.02,...,0.030000,0.010000,0.00,0.01,0.01,0.010000,0.01,0.00,0.020000,0.00
6,"81 Av norte y 7a calle poniente, #4119",0.000,0.000000,0.00,0.010000,0.00,0.010000,0.01,0.03,0.01,...,0.020000,0.020000,0.00,0.01,0.01,0.010000,0.00,0.00,0.020000,0.01
7,81 Ave. Sur Col. Escalon,0.000,0.010000,0.00,0.000000,0.01,0.000000,0.00,0.02,0.01,...,0.010000,0.010000,0.00,0.01,0.01,0.010000,0.01,0.00,0.020000,0.00
8,87 Av. Sur y Av. Los Almendros.,0.000,0.000000,0.00,0.000000,0.01,0.000000,0.00,0.02,0.01,...,0.010000,0.010000,0.00,0.01,0.01,0.010000,0.00,0.00,0.020000,0.00
9,89 Av Norte Escalon,0.000,0.010000,0.00,0.000000,0.01,0.000000,0.01,0.02,0.01,...,0.010000,0.010000,0.00,0.01,0.01,0.010000,0.00,0.00,0.020000,0.00


### After one-hot encoding the venues and categories of restaurants, frequency shall be meassured, to understand what types of bussiness are in the area.

In [205]:
#Frequency for Venue Category in Antiguo Cuscatlan.
num_topAC_venues = 5

for hood in AC_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = AC_grouped[AC_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_topAC_venues))
    print('\n')

----1a Calle Pte block P #21----
                venue  freq
0         Coffee Shop  0.07
1                Café  0.05
2         Pizza Place  0.04
3       Shopping Mall  0.04
4  Italian Restaurant  0.04


----1ra Calle Oriente----
                       venue  freq
0                Coffee Shop  0.07
1                 Restaurant  0.04
2              Shopping Mall  0.04
3         Italian Restaurant  0.04
4  Latin American Restaurant  0.04


----1° Calle Oriente----
                       venue  freq
0                Coffee Shop  0.07
1                 Restaurant  0.04
2              Shopping Mall  0.04
3         Italian Restaurant  0.04
4  Latin American Restaurant  0.04


----Antigua Vía Ferrocarril----
           venue  freq
0           Café  0.06
1    Coffee Shop  0.06
2          Hotel  0.05
3     Restaurant  0.04
4  Shopping Mall  0.04


----Antiguo Cuscatlan----
         venue  freq
0  Coffee Shop  0.07
1         Café  0.05
2  Pizza Place  0.04
3        Hotel  0.04
4   Restaurant  0.0

           venue  freq
0           Café  0.06
1    Coffee Shop  0.06
2         Bakery  0.04
3  Shopping Mall  0.04
4          Hotel  0.04


----Centro Comercial La Gran Via Local 11-B----
                       venue  freq
0                Coffee Shop  0.06
1  Latin American Restaurant  0.05
2                     Bakery  0.05
3                       Café  0.05
4              Shopping Mall  0.04


----Centro Comercial La Gran Vía----
                       venue  freq
0                Coffee Shop  0.06
1  Latin American Restaurant  0.05
2                     Bakery  0.05
3                       Café  0.05
4              Shopping Mall  0.04


----Centro Comercial Multiplaza----
                venue  freq
0                Café  0.06
1       Shopping Mall  0.05
2         Coffee Shop  0.05
3               Hotel  0.04
4  Italian Restaurant  0.04


----Centro de Estilo de Vida La Gran Vía----
                       venue  freq
0                       Café  0.05
1                     Bakery  

In [192]:
#Frequency for Venue Category in Santa Tecla.
num_topST_venues = 5

for hood in ST_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = ST_grouped[ST_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_topST_venues))
    print('\n')

----1 Calle Poniente----
                       venue  freq
0                        Bar  0.13
1  Latin American Restaurant  0.08
2                Pizza Place  0.07
3                     Bakery  0.03
4             Sandwich Place  0.03


----1 Calle Poniente, local 2-A----
                       venue  freq
0                        Bar  0.12
1  Latin American Restaurant  0.08
2                Pizza Place  0.07
3         Mexican Restaurant  0.03
4             Sandwich Place  0.03


----15 A Avenida Norte----
                       venue  freq
0                        Bar  0.15
1                Pizza Place  0.09
2                       Café  0.06
3  Latin American Restaurant  0.05
4                     Bakery  0.04


----16 Av Norte----
                       venue  freq
0                        Bar  0.16
1         Mexican Restaurant  0.05
2         Seafood Restaurant  0.05
3  Latin American Restaurant  0.05
4                       Park  0.04


----16 Avenida Norte----
                   

                       venue  freq
0                       Café  0.06
1              Shopping Mall  0.06
2                Pizza Place  0.05
3  Latin American Restaurant  0.05
4                     Bakery  0.05


----Centro Comercial La Skina----
                       venue  freq
0                        Bar  0.16
1                       Park  0.05
2         Mexican Restaurant  0.05
3         Seafood Restaurant  0.05
4  Latin American Restaurant  0.05


----Centro Comercial Plaza Borbon Local 1----
           venue  freq
0         Bakery  0.07
1           Café  0.06
2    Pizza Place  0.06
3  Shopping Mall  0.06
4   Burger Joint  0.04


----Centro Histórico de Santa Tecla----
                       venue  freq
0                        Bar  0.12
1  Latin American Restaurant  0.07
2                Pizza Place  0.07
3              Shopping Mall  0.05
4                       Café  0.04


----Centro comercial El Trebol----
                       venue  freq
0  Latin American Restaurant  0.06

In [212]:
#Frequency for Venue Category in San Salvador North.
num_topSS_venues = 5

for hood in Ss_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = Ss_grouped[Ss_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_topSS_venues))
    print('\n')

----11 Calle Poniente, Calle El Mirador----
                venue  freq
0         Pizza Place  0.06
1  Seafood Restaurant  0.05
2  Chinese Restaurant  0.04
3                Café  0.04
4  Mexican Restaurant  0.03


----3a Calle Poniente #5254 Lomas Verdes, Colonia Escalón----
                venue  freq
0         Pizza Place  0.06
1  Chinese Restaurant  0.05
2               Hotel  0.04
3                Café  0.04
4  Seafood Restaurant  0.03


----3a. Calle Poniente----
                venue  freq
0               Hotel  0.06
1          Restaurant  0.04
2  Italian Restaurant  0.04
3         Coffee Shop  0.04
4         Pizza Place  0.04


----3a. calle poniente # 4817----
                venue  freq
0                Café  0.05
1               Hotel  0.04
2         Pizza Place  0.04
3   German Restaurant  0.03
4  Italian Restaurant  0.03


----77 Av Norte #346 Col. Escalon----
         venue  freq
0         Café  0.06
1        Hotel  0.06
2  Pizza Place  0.04
3  Coffee Shop  0.03
4   Restau

                venue  freq
0         Pizza Place  0.06
1  Seafood Restaurant  0.05
2                Café  0.04
3  Chinese Restaurant  0.04
4   German Restaurant  0.03


----Centro Comercial Masferrrer, frente a Burger King Masferrer----
                venue  freq
0  Chinese Restaurant  0.05
1         Pizza Place  0.05
2               Hotel  0.04
3  Italian Restaurant  0.04
4                Café  0.03


----Centro Comercial Orión (frente ex-Kismet)----
                venue  freq
0               Hotel  0.05
1                Café  0.04
2  Italian Restaurant  0.04
3         Pizza Place  0.04
4          Restaurant  0.04


----Centro comercial Los Castaños----
                venue  freq
0         Pizza Place  0.06
1                Café  0.05
2  Seafood Restaurant  0.05
3  Chinese Restaurant  0.04
4        Liquor Store  0.03


----Col. Escalón 85 Av. Norte Pie. Dordely # 4407----
                venue  freq
0                Café  0.05
1          Restaurant  0.04
2         Pizza Place  0.0

### With a frequency, now we can determine commonness of a venue per neighborhood in the different municipalities.

In [206]:
def return_most_common_venues_AC(row, num_topAC_venues):
    row_categories_AC = row.iloc[1:]
    row_categories_sorted_AC = row_categories_AC.sort_values(ascending=False)
    
    return row_categories_sorted_AC.index.values[0:num_topAC_venues]

In [162]:
def return_most_common_venues_ST(row, num_topST_venues):
    row_categories_ST = row.iloc[1:]
    row_categories_sorted_ST = row_categories_ST.sort_values(ascending=False)
    
    return row_categories_sorted_ST.index.values[0:num_topST_venues]

In [213]:
def return_most_common_venues_SS(row, num_topSS_venues):
    row_categories_SS = row.iloc[1:]
    row_categories_sorted_SS = row_categories_SS.sort_values(ascending=False)
    
    return row_categories_sorted_SS.index.values[0:num_topSS_venues]

In [231]:
# Top 10 Most Common Venues Antiguo Cuscatlan
num_topAC_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_topAC_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted_AC = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted_AC['Neighborhood'] = AC_grouped['Neighborhood']

for ind in np.arange(AC_grouped.shape[0]):
    neighborhoods_venues_sorted_AC.iloc[ind, 1:] = return_most_common_venues_AC(AC_grouped.iloc[ind, :], num_topAC_venues)

neighborhoods_venues_sorted_AC.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,1a Calle Pte block P #21,Coffee Shop,Café,Restaurant,Pizza Place,Shopping Mall,Italian Restaurant,Sushi Restaurant,Latin American Restaurant,Bakery,Salad Place
1,1ra Calle Oriente,Coffee Shop,Latin American Restaurant,Italian Restaurant,Hotel,Pizza Place,Restaurant,Shopping Mall,Café,Bakery,Sushi Restaurant
2,1° Calle Oriente,Coffee Shop,Latin American Restaurant,Italian Restaurant,Hotel,Pizza Place,Restaurant,Café,Shopping Mall,Bakery,Salad Place
3,Antigua Vía Ferrocarril,Café,Coffee Shop,Hotel,Restaurant,Shopping Mall,Italian Restaurant,Bakery,Gastropub,Sushi Restaurant,Salad Place
4,Antiguo Cuscatlan,Coffee Shop,Café,Shopping Mall,Hotel,Pizza Place,Restaurant,Bakery,Italian Restaurant,Salad Place,Latin American Restaurant


In [248]:
neighborhoods_venues_sorted_AC

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,1a Calle Pte block P #21,Coffee Shop,Café,Restaurant,Pizza Place,Shopping Mall,Italian Restaurant,Sushi Restaurant,Latin American Restaurant,Bakery,Salad Place
1,1ra Calle Oriente,Coffee Shop,Latin American Restaurant,Italian Restaurant,Hotel,Pizza Place,Restaurant,Shopping Mall,Café,Bakery,Sushi Restaurant
2,1° Calle Oriente,Coffee Shop,Latin American Restaurant,Italian Restaurant,Hotel,Pizza Place,Restaurant,Café,Shopping Mall,Bakery,Salad Place
3,Antigua Vía Ferrocarril,Café,Coffee Shop,Hotel,Restaurant,Shopping Mall,Italian Restaurant,Bakery,Gastropub,Sushi Restaurant,Salad Place
4,Antiguo Cuscatlan,Coffee Shop,Café,Shopping Mall,Hotel,Pizza Place,Restaurant,Bakery,Italian Restaurant,Salad Place,Latin American Restaurant
5,Antiguo cuscatlan,Coffee Shop,Café,Hotel,Pizza Place,Latin American Restaurant,Restaurant,Bakery,Shopping Mall,Italian Restaurant,Salad Place
6,Av. La Revolucion 184,Mexican Restaurant,Coffee Shop,Pizza Place,Café,Italian Restaurant,Restaurant,Gastropub,Bakery,Shopping Mall,Burger Joint
7,Av. Rio Amazonas #3,Coffee Shop,Café,Hotel,Restaurant,Pizza Place,Gastropub,Italian Restaurant,Salad Place,Park,Mexican Restaurant
8,Ave. Antiguo Cuscatlan,Coffee Shop,Café,Hotel,Restaurant,Pizza Place,Gastropub,Italian Restaurant,Salad Place,Park,Mexican Restaurant
9,"Ave. La Revolucion, Col. San Benito",Coffee Shop,Mexican Restaurant,Italian Restaurant,Restaurant,Bakery,Pizza Place,Shopping Mall,Burger Joint,Café,Chinese Restaurant


In [232]:
num1ac = AC_merged['1st Most Common Venue'].value_counts()

In [239]:
dfnum1ac = pd.DataFrame(num1ac)
dfnum1ac = dfnum1ac.reset_index()
dfnum1ac.columns = ['Venue Type', 'Counts']
dfnum1ac

,Venue Type,Counts
0,Coffee Shop,32
1,Café,21
2,Latin American Restaurant,16
3,Mexican Restaurant,2
4,Pizza Place,2
5,Bakery,2


In [228]:
# Top 10 Most Common Venues Santa Tecla
num_topST_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_topST_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted_ST = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted_ST['Neighborhood'] = ST_grouped['Neighborhood']

for ind in np.arange(ST_grouped.shape[0]):
    neighborhoods_venues_sorted_ST.iloc[ind, 1:] = return_most_common_venues_ST(ST_grouped.iloc[ind, :], num_topST_venues)

neighborhoods_venues_sorted_ST.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,1 Calle Poniente,Bar,Latin American Restaurant,Pizza Place,Seafood Restaurant,Coffee Shop,Café,Burger Joint,Sandwich Place,Bakery,Department Store
1,"1 Calle Poniente, local 2-A",Bar,Latin American Restaurant,Pizza Place,Burger Joint,Coffee Shop,Sandwich Place,Bakery,Seafood Restaurant,Mexican Restaurant,Café
2,15 A Avenida Norte,Bar,Pizza Place,Café,Latin American Restaurant,Bakery,Mexican Restaurant,Sandwich Place,Coffee Shop,Chinese Restaurant,Fried Chicken Joint
3,16 Av Norte,Bar,Mexican Restaurant,Latin American Restaurant,Seafood Restaurant,Pizza Place,Park,Restaurant,Art Gallery,Convenience Store,Bakery
4,16 Avenida Norte,Bar,Mexican Restaurant,Latin American Restaurant,Seafood Restaurant,Pizza Place,Park,Restaurant,Art Gallery,Convenience Store,Bakery


In [229]:
num1st = ST_merged['1st Most Common Venue'].value_counts()

In [238]:
dfnum1st = pd.DataFrame(num1st)
dfnum1st = dfnum1st.reset_index()
dfnum1st.columns = ['Venue Type', 'Counts']
dfnum1st

,Venue Type,Counts
0,Bar,55
1,Café,5
2,Shopping Mall,2
3,Bakery,2
4,Pizza Place,2
5,Latin American Restaurant,1


In [214]:
# Top 10 Most Common Venues San Salvador (North)
num_topSs_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_topSS_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted_SS = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted_SS['Neighborhood'] = Ss_grouped['Neighborhood']

for ind in np.arange(Ss_grouped.shape[0]):
    neighborhoods_venues_sorted_SS.iloc[ind, 1:] = return_most_common_venues_SS(Ss_grouped.iloc[ind, :], num_topSS_venues)

neighborhoods_venues_sorted_SS.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,"11 Calle Poniente, Calle El Mirador",Pizza Place,Seafood Restaurant,Chinese Restaurant,Café,Spa
1,"3a Calle Poniente #5254 Lomas Verdes, Colonia ...",Pizza Place,Chinese Restaurant,Café,Hotel,Seafood Restaurant
2,3a. Calle Poniente,Hotel,Italian Restaurant,Café,Pizza Place,Restaurant
3,3a. calle poniente # 4817,Café,Hotel,Pizza Place,Italian Restaurant,Bistro
4,77 Av Norte #346 Col. Escalon,Hotel,Café,Pizza Place,Italian Restaurant,Shopping Mall


### Clustering the venues by neighborhood, will provide a better view of Venue categories by neighborhood.

In [172]:
kclusters = 5

AC_grouped_clustering = AC_grouped.drop('Neighborhood', 1)
ST_grouped_clustering = ST_grouped.drop('Neighborhood', 1)
Ss_grouped_clustering = Ss_grouped.drop('Neighborhood', 1)

kmeans_AC = KMeans(n_clusters=kclusters, random_state=0).fit(AC_grouped_clustering)
kmeans_ST = KMeans(n_clusters=kclusters, random_state=0).fit(ST_grouped_clustering)
kmeans_Ss = KMeans(n_clusters=kclusters, random_state=0).fit(Ss_grouped_clustering)

kmeans_AC.labels_[0:10]

array([0, 0, 0, 0, 0, 0, 2, 0, 0, 2])

In [173]:
kmeans_ST.labels_[0:10]

array([2, 2, 1, 4, 4, 1, 1, 1, 1, 1])

In [174]:
kmeans_Ss.labels_[0:10]

array([3, 0, 4, 4, 4, 4, 3, 4, 1, 4])

In [175]:
neighborhoods_venues_sorted_AC.insert(0, 'Cluster Labels', kmeans_AC.labels_)

In [176]:
neighborhoods_venues_sorted_ST.insert(0, 'Cluster Labels', kmeans_ST.labels_)

In [178]:
neighborhoods_venues_sorted_SS.insert(0, 'Cluster Labels', kmeans_Ss.labels_)

In [179]:
AC_merged = the_final_AC

AC_merged = AC_merged.join(neighborhoods_venues_sorted_AC.set_index('Neighborhood'), on='Neighborhood')

AC_merged.head()

,Venue,Category,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Biscuit Factory Café Bistro,Café,Calle Principal Antiguo Cuscatlan,13.672250,-89.240188,0,Coffee Shop,Café,Shopping Mall,Restaurant,Pizza Place,Latin American Restaurant,Italian Restaurant,Salad Place,Hotel,Bakery
1,Sushi King,Sushi Restaurant,Ave. Rio Amazonas,13.677708,-89.238621,0,Coffee Shop,Café,Hotel,Restaurant,Pizza Place,Italian Restaurant,Gastropub,Salad Place,Mexican Restaurant,Latin American Restaurant
2,Oasis: Jugos y Licuados,Juice Bar,Ave. Antiguo Cuscatlan,13.677677,-89.239972,0,Coffee Shop,Café,Hotel,Restaurant,Pizza Place,Gastropub,Italian Restaurant,Bakery,Salad Place,Mexican Restaurant
3,Green Dreams,Tea Room,hacienda los miranda,13.677662,-89.244861,0,Café,Coffee Shop,Hotel,Restaurant,Shopping Mall,Italian Restaurant,Bakery,Gastropub,Sushi Restaurant,Salad Place
4,Jardín Botánico La Laguna,Garden,Antiguo cuscatlan,13.670026,-89.246986,0,Coffee Shop,Café,Hotel,Pizza Place,Latin American Restaurant,Restaurant,Bakery,Shopping Mall,Italian Restaurant,Salad Place


In [180]:
ST_merged = newST

ST_merged = ST_merged.join(neighborhoods_venues_sorted_ST.set_index('Neighborhood'), on='Neighborhood')

ST_merged.head()

,Venue,Category,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Teklebab / The House of Kebab,Turkish Restaurant,5a. Ave norte,13.676083,-89.284610,2,Bar,Pizza Place,Latin American Restaurant,Shopping Mall,Café,Coffee Shop,Bakery,Mexican Restaurant,Sandwich Place,Seafood Restaurant
1,La Grotta Crossfit,Gym / Fitness Center,"5ta Calle Oriente 3-2, Santa Tecla",13.677385,-89.285073,2,Bar,Latin American Restaurant,Pizza Place,Café,Shopping Mall,Burger Joint,Coffee Shop,Bakery,Mexican Restaurant,Sandwich Place
2,Boost,Juice Bar,Plaza Merliot,13.678574,-89.276152,1,Bar,Pizza Place,Café,Latin American Restaurant,Bakery,Mexican Restaurant,Sandwich Place,Chinese Restaurant,Shopping Mall,Fried Chicken Joint
3,Paseo El Carmen,Plaza,1° Calle Pte.,13.675393,-89.286953,2,Bar,Latin American Restaurant,Pizza Place,Burger Joint,Coffee Shop,Sandwich Place,Bakery,Seafood Restaurant,Mexican Restaurant,Café
4,Museo Municipal Tecleno MUTE,Art Gallery,7 avenida norte,13.673568,-89.283809,2,Bar,Pizza Place,Latin American Restaurant,Shopping Mall,Café,Breakfast Spot,Mexican Restaurant,Bakery,Sandwich Place,Seafood Restaurant


In [181]:
Ss_merged = newSS

Ss_merged = Ss_merged.join(neighborhoods_venues_sorted_SS.set_index('Neighborhood'), on='Neighborhood')

Ss_merged.head()

,Venue,Category,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Sento Luxury Spa & Salon,Spa,3a. calle poniente # 4817,13.704912,-89.244470,4,Café,Hotel,Pizza Place,Italian Restaurant,Bistro
1,Vanilla Spoon,Frozen Yogurt Shop,"Centro Comercial Masferrrer, frente a Burger K...",13.703156,-89.248551,0,Chinese Restaurant,Pizza Place,Hotel,Italian Restaurant,German Restaurant
2,Jugueton Masferrer,Toy / Game Store,Plaza Masferrer,13.704420,-89.248532,0,Pizza Place,Chinese Restaurant,Hotel,Italian Restaurant,Burger Joint
3,Crêpe Lovers,Dessert Shop,Calle masferrer,13.705113,-89.249201,0,Pizza Place,Chinese Restaurant,Hotel,Café,Italian Restaurant
4,Taquería Los Tapatíos,Mexican Restaurant,Paseo General Escalon,13.703504,-89.247850,0,Pizza Place,Chinese Restaurant,Hotel,Italian Restaurant,Café


In [235]:
num1ss = Ss_merged['1st Most Common Venue'].value_counts()

In [237]:
dfnum1ss = pd.DataFrame(num1ss)
dfnum1ss = dfnum1ss.reset_index()
dfnum1ss.columns = ['Venue Type', 'Counts']
dfnum1ss

,Venue Type,Counts
0,Pizza Place,31
1,Café,12
2,Mexican Restaurant,11
3,Hotel,11
4,Coffee Shop,4
5,Chinese Restaurant,4
6,Italian Restaurant,3
7,Seafood Restaurant,2


## Map of Antiguo Cuscatlan Clustered by kNN

In [208]:
ACmap_clusters = folium.Map(location=[latitude_AC, longitude_AC], zoom_start=15)

# Color Scheme for clusters
xAC = np.arange(kclusters)
ysAC = [i + xAC + (i*xAC)**2 for i in range(kclusters)]
ACcolors_array = cm.rainbow(np.linspace(0, 1, len(ysAC)))
rainbow = [colors.rgb2hex(i) for i in ACcolors_array]

# add markers to the map
ACmarkers_colors = []
for lat, lon, poi, cluster in zip(AC_merged['Latitude'], AC_merged['Longitude'], AC_merged['Neighborhood'], AC_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(ACmap_clusters)
       
ACmap_clusters

### Analysis of Antiguo Cuscatlan on venue concurrency per neighborhood:

In [209]:
AC_merged.loc[AC_merged['Cluster Labels'] == 0, AC_merged.columns[[2] + list(range(5, AC_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Calle Principal Antiguo Cuscatlan,0,Coffee Shop,Café,Shopping Mall,Restaurant,Pizza Place,Latin American Restaurant,Italian Restaurant,Salad Place,Hotel,Bakery
1,Ave. Rio Amazonas,0,Coffee Shop,Café,Hotel,Restaurant,Pizza Place,Italian Restaurant,Gastropub,Salad Place,Mexican Restaurant,Latin American Restaurant
2,Ave. Antiguo Cuscatlan,0,Coffee Shop,Café,Hotel,Restaurant,Pizza Place,Gastropub,Italian Restaurant,Bakery,Salad Place,Mexican Restaurant
3,hacienda los miranda,0,Café,Coffee Shop,Hotel,Restaurant,Shopping Mall,Italian Restaurant,Bakery,Gastropub,Sushi Restaurant,Salad Place
4,Antiguo cuscatlan,0,Coffee Shop,Café,Hotel,Pizza Place,Latin American Restaurant,Restaurant,Bakery,Shopping Mall,Italian Restaurant,Salad Place
6,Clle. del Mediterráneo,0,Café,Coffee Shop,Hotel,Restaurant,Pizza Place,Italian Restaurant,Gastropub,Park,Mexican Restaurant,Latin American Restaurant
8,1a Calle Pte block P #21,0,Coffee Shop,Café,Restaurant,Pizza Place,Shopping Mall,Italian Restaurant,Sushi Restaurant,Latin American Restaurant,Bakery,Salad Place
9,Calle Cuscatlán Poniente,0,Coffee Shop,Restaurant,Pizza Place,Latin American Restaurant,Italian Restaurant,Shopping Mall,Hotel,Café,Salad Place,Bakery
13,Antiguo Cuscatlan,0,Coffee Shop,Café,Shopping Mall,Hotel,Pizza Place,Restaurant,Bakery,Italian Restaurant,Latin American Restaurant,Salad Place
15,1ra Calle Oriente,0,Coffee Shop,Latin American Restaurant,Italian Restaurant,Hotel,Pizza Place,Restaurant,Shopping Mall,Café,Bakery,Sushi Restaurant


In [210]:
AC_merged.loc[AC_merged['Cluster Labels'] == 1, AC_merged.columns[[2] + list(range(5, AC_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,Multiplaza,1,Café,Coffee Shop,Italian Restaurant,Hotel,Bakery,Restaurant,Shopping Mall,Sushi Restaurant,Salad Place,Seafood Restaurant
14,Carretera Panamericana,1,Café,Coffee Shop,Bakery,Shopping Mall,Hotel,Italian Restaurant,Sushi Restaurant,Mexican Restaurant,Salad Place,Restaurant
19,Carretera Panamericana,1,Café,Coffee Shop,Bakery,Shopping Mall,Hotel,Italian Restaurant,Sushi Restaurant,Mexican Restaurant,Salad Place,Restaurant
20,Centro Comercial La Gran Via Local 11-B,1,Coffee Shop,Bakery,Café,Latin American Restaurant,Shopping Mall,Mexican Restaurant,Hotel,Restaurant,Salad Place,Department Store
32,C.C La Gran Via,1,Bakery,Café,Coffee Shop,Shopping Mall,Latin American Restaurant,Salad Place,Mexican Restaurant,Sushi Restaurant,Hotel,Sandwich Place
35,SOHO Las Cascadas,1,Café,Bakery,Coffee Shop,Restaurant,Shopping Mall,Gastropub,Salad Place,Sushi Restaurant,Hotel,Italian Restaurant
38,Calle Panamericana,1,Coffee Shop,Bakery,Café,Latin American Restaurant,Shopping Mall,Mexican Restaurant,Hotel,Sushi Restaurant,Salad Place,Japanese Restaurant
40,Centro Comercial La Gran Vía,1,Coffee Shop,Bakery,Café,Latin American Restaurant,Shopping Mall,Mexican Restaurant,Japanese Restaurant,Hotel,Pizza Place,Salad Place
42,La Gran Via,1,Coffee Shop,Bakery,Café,Shopping Mall,Latin American Restaurant,Salad Place,Mexican Restaurant,Japanese Restaurant,Hotel,Pizza Place
44,Centro de Estilo de Vida La Gran Vía,1,Bakery,Café,Coffee Shop,Shopping Mall,Latin American Restaurant,Salad Place,Mexican Restaurant,Sushi Restaurant,Hotel,Sandwich Place


In [189]:
AC_merged.loc[AC_merged['Cluster Labels'] == 2, AC_merged.columns[[2] + list(range(5, AC_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,C.C Multiplaza,2,Café,Coffee Shop,Bistro,Hotel,Shopping Mall,Italian Restaurant,Sushi Restaurant,Mexican Restaurant,Salad Place,Restaurant
12,C.C Multiplaza,2,Café,Coffee Shop,Bistro,Hotel,Shopping Mall,Italian Restaurant,Sushi Restaurant,Mexican Restaurant,Salad Place,Restaurant
24,Centro Comercial Multiplaza,2,Café,Coffee Shop,Shopping Mall,Bistro,Hotel,Italian Restaurant,Sushi Restaurant,Mexican Restaurant,Salad Place,Restaurant
26,Multiplaza El Salvador,2,Café,Coffee Shop,Bistro,Hotel,Shopping Mall,Italian Restaurant,Sushi Restaurant,Mexican Restaurant,Salad Place,Restaurant
27,Avenida la Revolución,2,Café,Pizza Place,Coffee Shop,Bakery,Mexican Restaurant,Italian Restaurant,Hotel,Shopping Mall,Gastropub,Restaurant
28,Centro Comercial Multiplaza,2,Café,Coffee Shop,Shopping Mall,Bistro,Hotel,Italian Restaurant,Sushi Restaurant,Mexican Restaurant,Salad Place,Restaurant
30,Boulevard El Hipodromo,2,Café,Coffee Shop,Pizza Place,Bakery,Mexican Restaurant,Italian Restaurant,Hotel,Gastropub,Restaurant,Shopping Mall
58,Av. La Revolucion 184,2,Coffee Shop,Mexican Restaurant,Pizza Place,Café,Italian Restaurant,Restaurant,Gastropub,Bakery,Shopping Mall,Burger Joint
61,"Boulevar del Hipódromo, 443",2,Coffee Shop,Mexican Restaurant,Café,Shopping Mall,Italian Restaurant,Dessert Shop,Pizza Place,Hotel,Gastropub,Restaurant
66,Calle Circunvalacion,2,Coffee Shop,Mexican Restaurant,Shopping Mall,Italian Restaurant,Café,Dessert Shop,Pizza Place,Hotel,Gastropub,Restaurant


In [190]:
AC_merged.loc[AC_merged['Cluster Labels'] == 3, AC_merged.columns[[2] + list(range(5, AC_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
33,Col. Lomas de San Francisco,3,Latin American Restaurant,Pizza Place,Coffee Shop,Restaurant,Pharmacy,Market,Bar,Burger Joint,Hotel,Sandwich Place
37,Calle #1,3,Pizza Place,Hotel,Coffee Shop,Italian Restaurant,Seafood Restaurant,Bar,Restaurant,Fast Food Restaurant,Market,Sandwich Place
49,Calle Antigua A Huizucar,3,Pizza Place,Coffee Shop,Latin American Restaurant,Hotel,Restaurant,Seafood Restaurant,Market,Bar,Fast Food Restaurant,Sandwich Place
51,Lomas De San Francisco. Urb. Nueva Orleans #6,3,Latin American Restaurant,Coffee Shop,Pizza Place,Restaurant,Sandwich Place,Market,Bar,Gym,Café,Juice Bar


In [191]:
AC_merged.loc[AC_merged['Cluster Labels'] == 4, AC_merged.columns[[2] + list(range(5, AC_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Zona Industrial La Laguna,4,Latin American Restaurant,Coffee Shop,Café,Shopping Mall,Bakery,Salad Place,Restaurant,Pizza Place,Hotel,Italian Restaurant
10,Carr. Panamericana,4,Latin American Restaurant,Coffee Shop,Bakery,Café,Shopping Mall,Hotel,Pizza Place,Restaurant,Salad Place,Juice Bar
21,Blvd. Santa Elena,4,Latin American Restaurant,Coffee Shop,Café,Shopping Mall,Bakery,Pizza Place,Mexican Restaurant,Hotel,Restaurant,Salad Place
34,"Urb. Madre Selva, Blvd. Sur",4,Latin American Restaurant,Coffee Shop,Shopping Mall,Bakery,Salad Place,Restaurant,Café,Hotel,Supermarket,Department Store
41,Plaza Madero,4,Latin American Restaurant,Café,Shopping Mall,Coffee Shop,Pizza Place,Bakery,Mexican Restaurant,Hotel,Restaurant,Salad Place
46,"Esquina Calle 2 y calle 3, Centro de Estilo de...",4,Latin American Restaurant,Coffee Shop,Café,Shopping Mall,Bakery,Pizza Place,Japanese Restaurant,Hotel,Salad Place,Restaurant
47,C.C. Plaza Madero,4,Latin American Restaurant,Café,Shopping Mall,Coffee Shop,Pizza Place,Bakery,Mexican Restaurant,Hotel,Restaurant,Salad Place
50,Boulevard Santa Elena,4,Latin American Restaurant,Café,Shopping Mall,Coffee Shop,Pizza Place,Bakery,Mexican Restaurant,Hotel,Restaurant,Salad Place
52,Santa Elena,4,Latin American Restaurant,Coffee Shop,Café,Shopping Mall,Bakery,Pizza Place,Mexican Restaurant,Hotel,Restaurant,Salad Place
53,Plaza Sta Elena,4,Latin American Restaurant,Coffee Shop,Café,Shopping Mall,Pizza Place,Bakery,Hotel,Restaurant,Salad Place,Sandwich Place


## Map of Santa Tecla Clustered by kNN

In [185]:
STmap_clusters = folium.Map(location=[ST_latitude, ST_longitude], zoom_start=15)

# Color Scheme for clusters
xST = np.arange(kclusters)
ysST = [i + xST + (i*xST)**2 for i in range(kclusters)]
STcolors_array = cm.rainbow(np.linspace(0, 1, len(ysST)))
rainbow = [colors.rgb2hex(i) for i in STcolors_array]

# add markers to the map
STmarkers_colors = []
for lat, lon, poi, cluster in zip(ST_merged['Latitude'], ST_merged['Longitude'], ST_merged['Neighborhood'], ST_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(STmap_clusters)
       
STmap_clusters

### Analysis of Santa Tecla on venue concurrency per neighborhood:

In [215]:
ST_merged.loc[ST_merged['Cluster Labels'] == 0, ST_merged.columns[[2] + list(range(5, ST_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
13,Calle Chiltiupan,0,Café,Bar,Bakery,Pizza Place,Shopping Mall,Latin American Restaurant,Coffee Shop,Sandwich Place,Burger Joint,Supermarket
23,Ciudad Merliot,0,Pizza Place,Bar,Latin American Restaurant,Bakery,Café,Burger Joint,Sandwich Place,Shopping Mall,Soccer Field,Mexican Restaurant
24,Calle Chiltiupan,0,Café,Bar,Bakery,Pizza Place,Shopping Mall,Latin American Restaurant,Coffee Shop,Sandwich Place,Burger Joint,Supermarket
35,"boulevard orden de malta santa elena, antiguo ...",0,Bar,Pizza Place,Shopping Mall,Café,Latin American Restaurant,Bakery,Burger Joint,Supermarket,Sandwich Place,Hotel
37,Ciudad Merliot,0,Pizza Place,Bar,Latin American Restaurant,Bakery,Café,Burger Joint,Sandwich Place,Shopping Mall,Soccer Field,Mexican Restaurant
39,Calle Chiltiupan,0,Café,Bar,Bakery,Pizza Place,Shopping Mall,Latin American Restaurant,Coffee Shop,Sandwich Place,Burger Joint,Supermarket
43,Carretera a La Libertad y Boulevar Orden de Malta,0,Bar,Pizza Place,Shopping Mall,Latin American Restaurant,Bakery,Café,Supermarket,Coffee Shop,Sandwich Place,Salad Place
51,"Carretera panamericana, antiguo cuscatlan, el ...",0,Bar,Pizza Place,Café,Bakery,Latin American Restaurant,Shopping Mall,Sandwich Place,Burger Joint,Seafood Restaurant,Salad Place
62,Carretera Al Puerto de La Libertad,0,Bar,Pizza Place,Shopping Mall,Bakery,Café,Sandwich Place,Seafood Restaurant,Burger Joint,Supermarket,Latin American Restaurant
63,Calle Chiltiupan,0,Café,Bar,Bakery,Pizza Place,Shopping Mall,Latin American Restaurant,Coffee Shop,Sandwich Place,Burger Joint,Supermarket


In [216]:
ST_merged.loc[ST_merged['Cluster Labels'] == 1, ST_merged.columns[[2] + list(range(5, ST_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Plaza Merliot,1,Bar,Pizza Place,Café,Latin American Restaurant,Bakery,Mexican Restaurant,Sandwich Place,Chinese Restaurant,Shopping Mall,Fried Chicken Joint
5,"Ciudad Merliot, Santa Teresa",1,Bar,Pizza Place,Café,Latin American Restaurant,Bakery,Mexican Restaurant,Sandwich Place,Shopping Mall,Plaza,Fried Chicken Joint
6,Centro Comercial Chiltiupán,1,Bar,Pizza Place,Café,Latin American Restaurant,Bakery,Mexican Restaurant,Coffee Shop,Sandwich Place,Chinese Restaurant,Shopping Mall
7,"17 Ave Norte, #8, Col. Santa Teresa",1,Bar,Pizza Place,Café,Latin American Restaurant,Bakery,Mexican Restaurant,Sandwich Place,Chinese Restaurant,Seafood Restaurant,Fried Chicken Joint
8,Carretera Panamericana,1,Bar,Pizza Place,Bakery,Latin American Restaurant,Café,Supermarket,Sandwich Place,Shopping Mall,Seafood Restaurant,Gym
12,17 avenida norte,1,Bar,Pizza Place,Café,Latin American Restaurant,Bakery,Mexican Restaurant,Sandwich Place,Shopping Mall,Plaza,Fried Chicken Joint
14,Calle Chiltiupán,1,Bar,Pizza Place,Latin American Restaurant,Bakery,Café,Sandwich Place,Seafood Restaurant,Soccer Field,Shopping Mall,Chinese Restaurant
15,Calle Chiltiupán,1,Bar,Pizza Place,Latin American Restaurant,Bakery,Café,Sandwich Place,Seafood Restaurant,Soccer Field,Shopping Mall,Chinese Restaurant
22,Texaco Chiltiupan,1,Bar,Pizza Place,Café,Latin American Restaurant,Bakery,Mexican Restaurant,Sandwich Place,Shopping Mall,Plaza,Convenience Store
26,C.C Plaza Merliot,1,Bar,Pizza Place,Latin American Restaurant,Café,Bakery,Sandwich Place,Soccer Field,Supermarket,Mexican Restaurant,Plaza


In [217]:
ST_merged.loc[ST_merged['Cluster Labels'] == 2, ST_merged.columns[[2] + list(range(5, ST_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,5a. Ave norte,2,Bar,Pizza Place,Latin American Restaurant,Shopping Mall,Café,Coffee Shop,Bakery,Mexican Restaurant,Sandwich Place,Seafood Restaurant
1,"5ta Calle Oriente 3-2, Santa Tecla",2,Bar,Latin American Restaurant,Pizza Place,Café,Shopping Mall,Burger Joint,Coffee Shop,Bakery,Mexican Restaurant,Sandwich Place
3,1° Calle Pte.,2,Bar,Latin American Restaurant,Pizza Place,Burger Joint,Coffee Shop,Sandwich Place,Bakery,Seafood Restaurant,Mexican Restaurant,Café
4,7 avenida norte,2,Bar,Pizza Place,Latin American Restaurant,Shopping Mall,Café,Breakfast Spot,Mexican Restaurant,Bakery,Sandwich Place,Seafood Restaurant
9,Paseo El Carmen,2,Bar,Latin American Restaurant,Pizza Place,Café,Shopping Mall,Coffee Shop,Sandwich Place,Seafood Restaurant,Burger Joint,Bakery
10,7a. Avenida Norte,2,Bar,Pizza Place,Latin American Restaurant,Bakery,Café,Shopping Mall,Mexican Restaurant,Sandwich Place,Seafood Restaurant,Coffee Shop
11,1era. Calle Oriente 3-7,2,Bar,Pizza Place,Latin American Restaurant,Café,Shopping Mall,Burger Joint,Coffee Shop,Bakery,Mexican Restaurant,Sandwich Place
17,Calle José Ciriaco López,2,Bar,Latin American Restaurant,Pizza Place,Seafood Restaurant,Coffee Shop,Café,Burger Joint,Sandwich Place,Bakery,Department Store
18,"1 Calle Poniente, local 2-A",2,Bar,Latin American Restaurant,Pizza Place,Burger Joint,Coffee Shop,Sandwich Place,Bakery,Seafood Restaurant,Mexican Restaurant,Café
20,"Paseo El Carmen, entre 3 y 5 Av. Norte",2,Bar,Pizza Place,Latin American Restaurant,Café,Shopping Mall,Burger Joint,Coffee Shop,Bakery,Mexican Restaurant,Sandwich Place


In [218]:
ST_merged.loc[ST_merged['Cluster Labels'] == 3, ST_merged.columns[[2] + list(range(5, ST_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,Centro Comercial Plaza Borbon Local 1,3,Bakery,Pizza Place,Café,Shopping Mall,Burger Joint,Bar,Latin American Restaurant,Mexican Restaurant,Coffee Shop,Soccer Field
46,Merliot,3,Shopping Mall,Café,Pizza Place,Coffee Shop,Bakery,Latin American Restaurant,Bar,Park,Sandwich Place,Breakfast Spot
52,Centro Comercial El Trébol,3,Shopping Mall,Café,Latin American Restaurant,Bakery,Pizza Place,Bar,Coffee Shop,Sandwich Place,Park,Department Store
59,"Calle L-A, Poligono D #3 Jardines de Cuscatlan...",3,Café,Shopping Mall,Pizza Place,Coffee Shop,Bakery,Latin American Restaurant,Park,Breakfast Spot,Sandwich Place,Salad Place
60,Calle La Cañada,3,Bakery,Pizza Place,Café,Shopping Mall,Burger Joint,Bar,Latin American Restaurant,Mexican Restaurant,Coffee Shop,Soccer Field
64,Centro comercial El Trebol,3,Latin American Restaurant,Café,Shopping Mall,Bakery,Pizza Place,Coffee Shop,Sandwich Place,Park,Department Store,Breakfast Spot


In [219]:
ST_merged.loc[ST_merged['Cluster Labels'] == 4, ST_merged.columns[[2] + list(range(5, ST_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
16,Centro Comercial La Skina,4,Bar,Mexican Restaurant,Latin American Restaurant,Seafood Restaurant,Park,Pizza Place,Restaurant,Art Gallery,Convenience Store,Bakery
21,Pinares de Suiza,4,Bar,Latin American Restaurant,Seafood Restaurant,Park,Pizza Place,Shopping Mall,Bakery,Mexican Restaurant,Art Gallery,Convenience Store
25,CC La Skina,4,Bar,Latin American Restaurant,Seafood Restaurant,Pizza Place,Mexican Restaurant,Park,Restaurant,Art Gallery,Convenience Store,Bakery
27,Centro Comercial La Skina,4,Bar,Mexican Restaurant,Latin American Restaurant,Seafood Restaurant,Park,Pizza Place,Restaurant,Art Gallery,Convenience Store,Bakery
32,16 Avenida Norte,4,Bar,Mexican Restaurant,Latin American Restaurant,Seafood Restaurant,Pizza Place,Park,Restaurant,Art Gallery,Convenience Store,Bakery
44,16 Av Norte,4,Bar,Mexican Restaurant,Latin American Restaurant,Seafood Restaurant,Pizza Place,Park,Restaurant,Art Gallery,Convenience Store,Bakery
48,9na calle ponientepoligonoc--7casa #2 residenc...,4,Bar,Latin American Restaurant,Pizza Place,Park,Bakery,Seafood Restaurant,Sandwich Place,Mexican Restaurant,Burger Joint,Restaurant
49,Calle Real,4,Bar,Mexican Restaurant,Latin American Restaurant,Seafood Restaurant,Pizza Place,Park,Restaurant,Art Gallery,Convenience Store,Bakery
55,Frente Mercado Dueñas,4,Bar,Latin American Restaurant,Pizza Place,Burger Joint,Seafood Restaurant,Bakery,Park,Mexican Restaurant,Sandwich Place,Convenience Store


## Map of San Salvadore Clustered by kNN

In [186]:
SSmap_clusters = folium.Map(location=[SS_latitude, SS_longitude], zoom_start=15)

# Color Scheme for clusters
xSS = np.arange(kclusters)
ysSS = [i + xSS + (i*xSS)**2 for i in range(kclusters)]
colors_arraySS = cm.rainbow(np.linspace(0, 1, len(ysSS)))
rainbow = [colors.rgb2hex(i) for i in colors_arraySS]

# add markers to the map
markers_colorsSS = []
for lat, lon, poi, cluster in zip(Ss_merged['Latitude'], Ss_merged['Longitude'], Ss_merged['Neighborhood'], Ss_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(SSmap_clusters)
       
SSmap_clusters

### Analysis of San Salvador on venue concurrency per neighborhood:

In [220]:
Ss_merged.loc[Ss_merged['Cluster Labels'] == 0, Ss_merged.columns[[2] + list(range(5, Ss_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
1,"Centro Comercial Masferrrer, frente a Burger K...",0,Chinese Restaurant,Pizza Place,Hotel,Italian Restaurant,German Restaurant
2,Plaza Masferrer,0,Pizza Place,Chinese Restaurant,Hotel,Italian Restaurant,Burger Joint
3,Calle masferrer,0,Pizza Place,Chinese Restaurant,Hotel,Café,Italian Restaurant
4,Paseo General Escalon,0,Pizza Place,Chinese Restaurant,Hotel,Italian Restaurant,Café
5,Colonia Escalon,0,Pizza Place,Chinese Restaurant,Café,Seafood Restaurant,Italian Restaurant
6,Calle Maximo Jerez,0,Pizza Place,Chinese Restaurant,Italian Restaurant,Bistro,German Restaurant
7,Colonia Escalon,0,Pizza Place,Chinese Restaurant,Café,Seafood Restaurant,Italian Restaurant
13,Paseo General Escalon,0,Pizza Place,Chinese Restaurant,Hotel,Italian Restaurant,Café
19,"Plaza Barcelona, Colonia Escalón",0,Pizza Place,Chinese Restaurant,Café,Seafood Restaurant,Mexican Restaurant
22,"3a Calle Poniente #5254 Lomas Verdes, Colonia ...",0,Pizza Place,Chinese Restaurant,Café,Hotel,Seafood Restaurant


In [221]:
Ss_merged.loc[Ss_merged['Cluster Labels'] == 1, Ss_merged.columns[[2] + list(range(5, Ss_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
8,Plaza Las Azaleas,1,Mexican Restaurant,Italian Restaurant,Pizza Place,Chinese Restaurant,Restaurant
9,Centro Comercial Las Azaleas,1,Mexican Restaurant,Italian Restaurant,Pizza Place,Chinese Restaurant,Restaurant
15,"Plaza L'Abero, Local 1A",1,Chinese Restaurant,Mexican Restaurant,Italian Restaurant,Hotel,Restaurant
16,87 Av. Sur y Av. Los Almendros.,1,Italian Restaurant,Café,Mexican Restaurant,Chinese Restaurant,Pizza Place
17,Calle y Colonia La Mascota No.942,1,Chinese Restaurant,Mexican Restaurant,Italian Restaurant,Restaurant,Hotel
23,"Calle La Mascota #7-A, Col. Maquilishuat",1,Italian Restaurant,Chinese Restaurant,Hotel,Mexican Restaurant,Pizza Place
27,Calle La Mascota #803,1,Mexican Restaurant,Italian Restaurant,Pizza Place,Chinese Restaurant,Restaurant
29,Plaza Azaleas,1,Mexican Restaurant,Italian Restaurant,Pizza Place,Chinese Restaurant,Restaurant
39,"Plaza L´Albero , Esquina Final Calle La Mascot...",1,Mexican Restaurant,Chinese Restaurant,Italian Restaurant,Pizza Place,Restaurant


In [222]:
Ss_merged.loc[Ss_merged['Cluster Labels'] == 2, Ss_merged.columns[[2] + list(range(5, Ss_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
26,"Blv. El Hipodromo #582, San Benito",2,Mexican Restaurant,Coffee Shop,Pizza Place,Italian Restaurant,Hotel
43,"Final Avenida La Revolución, Colonia San Benito",2,Mexican Restaurant,Coffee Shop,Pizza Place,Italian Restaurant,Hotel
44,"Colonia San Benito, Blv Del Ipodromo, 605",2,Mexican Restaurant,Italian Restaurant,Hotel,Coffee Shop,Pizza Place
45,Final Avenida la Revolución,2,Coffee Shop,Mexican Restaurant,Pizza Place,Hotel,Italian Restaurant
48,Blvd. Del Hipódromo 544,2,Mexican Restaurant,Pizza Place,Italian Restaurant,Restaurant,Hotel
54,Calle Circunvalación intersección con pasaje I...,2,Mexican Restaurant,Coffee Shop,Pizza Place,Italian Restaurant,Hotel
66,Av. La Revolucion 184,2,Coffee Shop,Mexican Restaurant,Italian Restaurant,Pizza Place,Café
67,"Ave. La Revolucion, Col. San Benito",2,Coffee Shop,Mexican Restaurant,Café,Italian Restaurant,Burger Joint
68,Av. Magnolias y Boulevard del Hipod,2,Mexican Restaurant,Coffee Shop,Italian Restaurant,Café,Hotel
73,Blvd del Hipodromo #13,2,Hotel,Mexican Restaurant,Italian Restaurant,Café,Burger Joint


In [223]:
Ss_merged.loc[Ss_merged['Cluster Labels'] == 3, Ss_merged.columns[[2] + list(range(5, Ss_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
10,"81 Av norte y 7a calle poniente, #4119",3,Pizza Place,Seafood Restaurant,Café,Chinese Restaurant,Mexican Restaurant
11,"Centro Comercial Los Castaños, Av. Masferrer.",3,Pizza Place,Seafood Restaurant,Chinese Restaurant,Café,Liquor Store
12,"Avenida Masferrer Norte, Comercial Barcelona",3,Pizza Place,Café,Seafood Restaurant,Chinese Restaurant,Burger Joint
18,Centro comercial Los Castaños,3,Pizza Place,Seafood Restaurant,Café,Chinese Restaurant,Liquor Store
20,Calle El Mirador-Escalón,3,Seafood Restaurant,Pizza Place,Coffee Shop,Café,Mexican Restaurant
28,Centro Comercial Los Castaños Av. Alberto Masf...,3,Pizza Place,Seafood Restaurant,Café,Chinese Restaurant,Liquor Store
30,9 C poniente bis,3,Seafood Restaurant,Pizza Place,Coffee Shop,Café,Mexican Restaurant
32,Final Paseo General Escalón,3,Pizza Place,Chinese Restaurant,Liquor Store,Seafood Restaurant,Spa
34,Plaza Los Castaños,3,Pizza Place,Seafood Restaurant,Chinese Restaurant,Café,Spa
41,"Hoteles Y Servicios Turisticos, S.A, 89 Ave. N...",3,Pizza Place,Coffee Shop,Seafood Restaurant,Café,Chinese Restaurant


In [224]:
Ss_merged.loc[Ss_merged['Cluster Labels'] == 4, Ss_merged.columns[[2] + list(range(5, Ss_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,3a. calle poniente # 4817,4,Café,Hotel,Pizza Place,Italian Restaurant,Bistro
14,Torre Futura,4,Café,Pizza Place,Italian Restaurant,Bistro,Mexican Restaurant
21,89 Av Norte Escalon,4,Café,Italian Restaurant,Hotel,Pizza Place,Restaurant
24,Col. Escalón 85 Av. Norte Pie. Dordely # 4407,4,Café,Italian Restaurant,Pizza Place,Restaurant,Hotel
31,91 Avenida Norte #532,4,Café,Pizza Place,Seafood Restaurant,Mexican Restaurant,Burger Joint
35,Plaza Futura,4,Café,Pizza Place,Seafood Restaurant,Chinese Restaurant,Bistro
36,"79 Av Norte Y 3 Calle Poniente,colonia Escalon",4,Hotel,Italian Restaurant,Café,Pizza Place,Restaurant
38,81 Ave. Sur Col. Escalon,4,Italian Restaurant,Café,Hotel,Pizza Place,Mexican Restaurant
42,Plaza Futura,4,Café,Pizza Place,Seafood Restaurant,Chinese Restaurant,Bistro
47,Plaza Futura,4,Café,Pizza Place,Seafood Restaurant,Chinese Restaurant,Bistro


## Results and Discussion

After the analysis of venue type distribution through MASS, it becomes clear that three main municipalities excel when thinking about opening a business in the area. Antiguo Cuscatlan, Santa Tecla, and San Salvador (North). kNN machine learning algorithm, frequencies and simple counts of the area also reveal which type of venue is most concurrent in the different municipalities:

**Antiguo Cuscatlan**: Coffee Shops or Café

**Santa Tecla**: Bars

**San Salvador**: Pizza Place

### Most concurred areas are also shown by the kNN algorythm: 

**Antiguo Cuscatlan**: Neighborhoods in Cluster 0

**Santa Tecla**: Neighborhoods in  Cluster 2

**San Salvador**: Neighborhoods in  Cluster 0 and 4

The recommendation would be to open a Cafe, a Bar or a Pizza place since it was shown by kNN that Pizza, Bars and Cafes are still concurrent type of venues in other areas. It might be an idea worth considering but opening a Bar that specializes in cooking pizza and has advanced coffee cooking and serving techniques would be a safe bet. It is also recommended to expand the study of the competition in the areas and do a SWOT analysis on them, to know how to compete against other venues.

## Conclusion

The study reflects clearly where and how to invest money in MASS. El Salvador does have FTA with the United States, UE, Japan and many other countries, these treaties help investors in many ways; El Salvador does count with a government entity that assists for all foreign investment (PROESA). Now then, this study can be taken by anyone interested who wants to invest in the MASS area, not only foreigners. In the past, El Salvador has had problems with security, but in these last year, crime rates have hit a new low, stimulating for investment. 

This study does not correlate crime data, and real state pricing, the country does not have free access to this type of information. Therefore, other studies in terms of security and real state should be considered to build a complete business plan.